In [1]:
import pandas as pd
import numpy as np
import re 
import warnings
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

from unidecode import unidecode
from nltk.corpus import stopwords

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

## Dataset web scraped from Vinissimus.com

In [2]:
vinissimus = pd.read_csv("../01-initial_data/webscraping/wine_data.csv")
vinissimus.sample(10)

,name,price,bottle,type_wine,region,grapes,producer,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,alergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url
3844,Cara Nord Negre 2020,£13.80,/ 0.75 L btl,Red wine,Conca de Barberà (Spain),42% Garnacha38% Syrah20% Monastrell,Cara Nord,The Cara Nord (meaning 'the north face of a mo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve between 14ºC and 15ºC,NaN,Contains sulfites,14.5%,NaN,NaN,Stainless steel / Cement,Between 5 and 6 months,French oak,NaN,NaN,NaN,NaN,Between 20 and 25 years,Limestone / Clay / Slate,NaN,NaN,NaN,4.4/5,9 reviews,90 PK,NaN,2015.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/cara-nord-n...
6850,Carles Andreu Rosat Trepat Brut,£14.00,NaN,Cava,DO Cava (Spain),100% Trepat,Celler Carles Andreu,"Cellers Carles Andreu started up in 1991, with...",NaN,Cherry red / Fine and slow-rising bubbles,Fruity / Red fruit / Liquorice / Fruit jellies,Dry / Balanced acidity / Vinous / Pleasant,50.0,30.0,60.0,NaN,NaN,Serve between 8ºC and 10ºC,NaN,Contains sulfites,11.5%,NaN,NaN,18 months,NaN,NaN,NaN,NaN,NaN,NaN,Continental with Mediterranean influence,NaN,NaN,NaN,NaN,4.8/5,4 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/carles-andr...
1437,AA Parvus Chardonnay 2022,£13.20,/ 0.75 L btl,White wine,Alella (Spain),85% Chardonnay15% Pensal Blanca,Alta Alella,Chardonnay of pale yellow color and golden ref...,The family-run project of Josep Maria Pujol-Bu...,Deep yellow / Clean / Bright,Fruit aromas / Tropical fruits / Mango / Papay...,Oily / Unctuous / Balanced / Long finish / Ple...,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,Contains sulfites,13.0%,NaN,NaN,Wood,4 months,New,French oak,NaN,NaN,ORGANIC,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/aa-parvus-c...
5157,Attis Pedral 2016,£33.95,NaN,Red wine,Rías Baixas (Spain),100% Pedral,Attis Bodegas y Viñedos,The Attis winery was founded in 2000 by brothe...,NaN,Cherry red / Intense,Subtle / Delicate / Complex / Fruit aromas / R...,Rounded / Velvety tannins / Elegant / Good aci...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,12.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PEÑÍN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,90 PN,2016.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/attis-pedral/
5816,Barcolobo Lacrimae Rerum 2021,£17.50,NaN,Oak-aged Rosé Wine,Castile and León (Spain),100% Tempranillo,Finca la Rinconada,NaN,NaN,Pale pink / Bright / Bluish glints,Subtle / Floral scents / Violets / Roses / Fru...,Elegant / Good acidity / Fresh / Fruit-forward...,NaN,NaN,NaN,NaN,NaN,8ºC,NaN,Contains sulfites,13.0%,NaN,NaN,15 years,Sandy loam / Pebbles,Continental with Atlantic influence,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/barcolobo-l...
1635,Casa Vella d'Espiells 2018,£15.60,/ 0.75 L btl,Red wine,Penedès (Spain),100% Cabernet Sauvignon,Juvé y Camps,Ideal for accompanying dishes with roast meats...,The history of Juvé & Camps goes all the way b...,Ruby red / Intense,Profound / Higly aromatic / Complex / Subtle /...,Concentrated / Unctuous / Opulent,NaN,NaN,NaN,NaN,NaN,Serve between 16ºC and 17ºC,NaN,Contains sulfites,15.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/casa-vella-...
521,Attis 2022,£19.20,/ 0.75 

In [3]:
vinissimus.columns

Index(['name', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'producer',
       'wine_description', 'tasting_descr', 'tasting_view', 'tasting_bouquet',
       'tasting_mouth', 'light_bold_perc', 'smooth_tannic_perc',
       'dry_sweet_perc', 'soft_acidic_perc', 'serving_storage_conditions',
       'serve_temperature', 'food_pairing', 'alergens', 'alcohol_content',
       'production', 'winemaking_info', 'vessel_material', 'aging_period',
       'casks_age', 'casks_wood', 'more_winemaking_info', 'vineyards',
       'type_agriculture', 'vineyard_age', 'soil_type', 'climate',
       'orientation', 'surface', 'customer_reviews', 'customer_reviews_number',
       'last_score_parker', 'last_score_penin', 'last_score_year', 'image',
       'url'],
      dtype='object')

In [4]:
# Change the name of some columns

vinissimus = vinissimus.rename(columns={"name": "wine", "producer": "winery", "alergens": "allergens"})
vinissimus.columns

Index(['wine', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'winery',
       'wine_description', 'tasting_descr', 'tasting_view', 'tasting_bouquet',
       'tasting_mouth', 'light_bold_perc', 'smooth_tannic_perc',
       'dry_sweet_perc', 'soft_acidic_perc', 'serving_storage_conditions',
       'serve_temperature', 'food_pairing', 'allergens', 'alcohol_content',
       'production', 'winemaking_info', 'vessel_material', 'aging_period',
       'casks_age', 'casks_wood', 'more_winemaking_info', 'vineyards',
       'type_agriculture', 'vineyard_age', 'soil_type', 'climate',
       'orientation', 'surface', 'customer_reviews', 'customer_reviews_number',
       'last_score_parker', 'last_score_penin', 'last_score_year', 'image',
       'url'],
      dtype='object')

In [5]:
# Make a copy of the original dataframe before the cleaning

vinissimus_original = vinissimus.copy()

### Feature types

In [6]:
vinissimus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7016 entries, 0 to 7015
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   wine                        7016 non-null   object 
 1   price                       7016 non-null   object 
 2   bottle                      4548 non-null   object 
 3   type_wine                   7016 non-null   object 
 4   region                      7016 non-null   object 
 5   grapes                      7016 non-null   object 
 6   winery                      7016 non-null   object 
 7   wine_description            3625 non-null   object 
 8   tasting_descr               1470 non-null   object 
 9   tasting_view                4974 non-null   object 
 10  tasting_bouquet             4974 non-null   object 
 11  tasting_mouth               4974 non-null   object 
 12  light_bold_perc             435 non-null    float64
 13  smooth_tannic_perc          435 n

Many columns that should be numerical appear as object. We will solve that during the cleaning of each column.

### Duplicates

In [7]:
vinissimus.duplicated().value_counts()

False    5355
True     1661
dtype: int64

In [8]:
vinissimus = vinissimus.drop_duplicates()
vinissimus.duplicated().value_counts()

False    5355
dtype: int64

### NaN values

In [9]:
vinissimus.isna().sum()

wine                             0
price                            0
bottle                        1773
type_wine                        0
region                           0
grapes                           0
winery                           0
wine_description              2550
tasting_descr                 4180
tasting_view                  1532
tasting_bouquet               1532
tasting_mouth                 1532
light_bold_perc               4974
smooth_tannic_perc            4974
dry_sweet_perc                4974
soft_acidic_perc              5098
serving_storage_conditions    5170
serve_temperature             1643
food_pairing                  5145
allergens                        0
alcohol_content                 38
production                    4648
winemaking_info               5229
vessel_material               2901
aging_period                  3424
casks_age                     4306
casks_wood                    5026
more_winemaking_info          5308
vineyards           

In [10]:
nulls = pd.DataFrame(vinissimus.isna().sum()*100/len(vinissimus), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)

,percentage
more_winemaking_info,99.122316
vineyards,98.618114
surface,98.468721
winemaking_info,97.647059
serving_storage_conditions,96.545285
food_pairing,96.078431
soft_acidic_perc,95.200747
orientation,95.088702
casks_wood,93.856209
dry_sweet_perc,92.885154


There are a lot columns with high proportion of NaN values. This was expected since many of the wines in the webpage didn't have all the info. <br>
We will deal with the NaN values in each feature if it's needed. For now, we will use them to help decide in the feature selection. 

### Feature selection (first round)

**Columns to keep:**
- Less than 50% of NaNs.
- Some columns, like the ones related with the Parker and Peñín scores, or the type of agriculture, have a high number of NaNs, but that was expected (only a few wines have gained scores, only a few wines are organic/biodynamic). 
- Tasting-related columns: we will used them for the recommender.

In [11]:
vinissimus.columns 

Index(['wine', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'winery',
       'wine_description', 'tasting_descr', 'tasting_view', 'tasting_bouquet',
       'tasting_mouth', 'light_bold_perc', 'smooth_tannic_perc',
       'dry_sweet_perc', 'soft_acidic_perc', 'serving_storage_conditions',
       'serve_temperature', 'food_pairing', 'allergens', 'alcohol_content',
       'production', 'winemaking_info', 'vessel_material', 'aging_period',
       'casks_age', 'casks_wood', 'more_winemaking_info', 'vineyards',
       'type_agriculture', 'vineyard_age', 'soil_type', 'climate',
       'orientation', 'surface', 'customer_reviews', 'customer_reviews_number',
       'last_score_parker', 'last_score_penin', 'last_score_year', 'image',
       'url'],
      dtype='object')

In [12]:
keep_cols = list(nulls[nulls["percentage"]<50].index)
# keep_cols

In [13]:
more_cols_to_keep = ['tasting_descr', 'food_pairing', 'type_agriculture', 'last_score_parker',
                    'last_score_penin', 'last_score_year']

keep_cols = keep_cols + more_cols_to_keep
keep_cols

['wine',
 'price',
 'bottle',
 'type_wine',
 'region',
 'grapes',
 'winery',
 'wine_description',
 'tasting_view',
 'tasting_bouquet',
 'tasting_mouth',
 'serve_temperature',
 'allergens',
 'alcohol_content',
 'customer_reviews',
 'customer_reviews_number',
 'image',
 'url',
 'tasting_descr',
 'food_pairing',
 'type_agriculture',
 'last_score_parker',
 'last_score_penin',
 'last_score_year']

In [14]:
vinissimus_short = vinissimus[keep_cols]
vinissimus_short.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,allergens,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year
0,Ultreia Saint Jacques 2021,£16.20,/ 0.75 L btl,Red wine,Bierzo (Spain),100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,Contains sulfites,13.5%,4.2/5,217 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,NaN,93-94 PK,NaN,2020.0
1,Blanc Pescador,£7.80,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,Contains sulfites,11.5%,4.4/5,45 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,NaN,NaN,NaN,NaN
2,Bach Extrísimo Blanco Semidulce 2021,£9.40,/ 0.75 L btl,White wine,Catalunya (Spain),Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,Contains sulfites,11.5%,4.8/5,95 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,NaN,NaN,NaN,NaN
3,José Pariente Verdejo 2022,£16.20,/ 0.75 L btl,White wine,Rueda (Spain),100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,Contains sulfites,13.0%,4.4/5,147 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,NaN,NaN,NaN,2021.0
4,Coto de Imaz Reserva 2018,£13.00,/ 0.75 L btl,Red wine,Rioja (Spain),100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,Contains sulfites,13.5%,4.4/5,94 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,SUCKLING,NaN,NaN,2018.0


Now we will go one by one cleaning the columns. 

### Wine names

In [15]:
# Create new column with year

vinissimus["year"] = vinissimus["wine"].apply(lambda x: x[-4:]) # takes the last 4 characters
vinissimus["year"].value_counts()

2021    781
2019    740
2020    691
2018    612
2022    509
       ... 
2023      1
León      1
C.B.      1
lona      1
yrah      1
Name: year, Length: 202, dtype: int64

In [16]:
vinissimus["year"] = vinissimus["year"].apply(lambda x: int(x) if x.isdigit() else np.nan) # if the last characters were not numbers, change to NaN
vinissimus["year"].value_counts(dropna=False)

2021.0    781
2019.0    740
2020.0    691
2018.0    612
NaN       521
2022.0    509
2017.0    398
2016.0    344
2015.0    241
2014.0    171
2013.0     89
2012.0     68
2011.0     51
2010.0     46
2009.0     23
2008.0     11
2005.0     10
2007.0     10
2006.0      7
2004.0      5
2001.0      4
1999.0      2
2003.0      2
1926.0      2
1927.0      1
1951.0      1
1996.0      1
1952.0      1
1968.0      1
1865.0      1
2002.0      1
1989.0      1
1847.0      1
1872.0      1
1981.0      1
1973.0      1
1958.0      1
1980.0      1
1934.0      1
2023.0      1
1964.0      1
Name: year, dtype: int64

In [17]:
vinissimus["year"] = vinissimus["year"].astype('Int64') # "Int64" converts the column to nullable integers (NaN is float)
vinissimus["year"].value_counts(dropna=False)

2021    781
2019    740
2020    691
2018    612
<NA>    521
2022    509
2017    398
2016    344
2015    241
2014    171
2013     89
2012     68
2011     51
2010     46
2009     23
2008     11
2007     10
2005     10
2006      7
2004      5
2001      4
1926      2
2003      2
1999      2
1968      1
1964      1
1847      1
1934      1
1927      1
2023      1
1980      1
1958      1
1973      1
1981      1
1989      1
1872      1
1951      1
2002      1
1865      1
1996      1
1952      1
Name: year, dtype: Int64

In [18]:
keep_cols.append("year")

### Price

In [19]:
vinissimus["price"].value_counts(dropna=False)

£13.40     108
£12.40      91
£11.60      79
£15.60      68
£14.20      66
          ... 
£33.10       1
£113.75      1
£346.85      1
£56.65       1
£135.20      1
Name: price, Length: 779, dtype: int64

In [20]:
# Remove extra characters ("£" and ",")

vinissimus["price"] = vinissimus["price"].str.replace("£", "").str.replace(",", "").astype(float)
vinissimus["price"].describe()

count    5355.000000
mean       28.194118
std        65.830908
min         4.400000
25%        13.600000
50%        18.500000
75%        27.500000
max      2143.100000
Name: price, dtype: float64

### Bottle size

In [21]:
# There are 1773 NaN values in this column, this is too much and probably due to a problem in webscraping
# Let's extract those wines

bottle_nan = vinissimus[vinissimus["bottle"].isna()]
bottle_nan.to_csv("bottle_nan.csv", index=False)

In [22]:
# Import the new data from webscraping

bottle = pd.read_csv("../01-initial_data/webscraping/bottle_final.csv")
bottle.sample(5)

,url,bottle
1267,https://www.vinissimus.com/en/wine/huellas/,/ 0.75 L btl
985,https://www.vinissimus.com/en/wine/atalaque-ga...,/ 0.75 L btl
859,https://www.vinissimus.com/en/wine/fedellos-do...,/ 0.75 L btl
736,https://www.vinissimus.com/en/wine/sonsierra-t...,/ 0.75 L btl
2076,https://www.vinissimus.com/en/wine/marques-de-...,/ 0.75 L btl


In [23]:
bottle.duplicated().value_counts()

False    1717
True      682
dtype: int64

In [24]:
bottle = bottle.drop_duplicates()
bottle.duplicated().value_counts()

False    1717
dtype: int64

In [25]:
vinissimus.shape

(5355, 43)

In [26]:
# Merge dataframes
vinissimus = vinissimus.merge(bottle[["url", "bottle"]], how="left", left_on="url", right_on="url")
vinissimus.head()

,wine,price,bottle_x,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,bottle_y
0,Ultreia Saint Jacques 2021,16.2,/ 0.75 L btl,Red wine,Bierzo (Spain),100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5%,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,NaN,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2/5,217 reviews,93-94 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021,NaN
1,Blanc Pescador,7.8,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4/5,45 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,NaN
2,Bach Extrísimo Blanco Semidulce 2021,9.4,/ 0.75 L btl,White wine,Catalunya (Spain),Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5%,NaN,NaN,6 months,American oak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8/5,95 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,2021,NaN
3,José Pariente Verdejo 2022,16.2,/ 0.75 L btl,White wine,Rueda (Spain),100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,40.0,30.0,70.0,NaN,"Due to its freshness and acidity, it works ver...",Serve between 6ºC and 8ºC,FishSaladsStarters and suppersVegetablesSoft c...,Contains sulfites,13.0%,NaN,It is produced using grapes from both the fami...,Stainless steel / Concrete / Wood,4 months,NaN,NaN,NaN,"Bodegas José Pariente started up in the 1960s,...",NaN,Stony / Sand,Continental with Atlantic influence,NaN,NaN,NaN,4.4/5,147 reviews,NaN,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,2022,NaN
4,Coto de Imaz Reserva 2018,13.0,/ 0.75 L btl,Red wine,Rioja (Spain),100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,70.0,60.0,20.0

In [27]:
vinissimus.shape

(5355, 44)

In [28]:
bottle_nulls = vinissimus[vinissimus["bottle_x"].isna()]
bottle_nulls.head()

,wine,price,bottle_x,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,bottle_y
88,Viña Tondonia Reserva Blanco 2012,74.2,NaN,White wine,Rioja (Spain),90% Viura10% Malvasia,Bodegas R. López de Heredia,Tondonia whites are much more than classics. T...,R. López de Heredia (Viña Tondonia) is one of ...,Pale amber,Fine / Complex / Evolving / Crystallised fruit...,Smooth / Firm / Persistent,NaN,NaN,NaN,NaN,NaN,Serve between 14ºC and 16ºC,NaN,Contains sulfites,12.5%,"35,000 bottles",NaN,72 months,Used,American oak,Unfiltered,NaN,NaN,PARKER,100 years,Alluvial / Limestone,Atlantic,110.00 hectares,NaN,4.7/5,36 reviews,95 PK,NaN,2012.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-tondon...,2012,/ 0.75 L btl
776,Ipsis Chardonnay 2018,11.4,NaN,White wine,Tarragona (Spain),100% Chardonnay,Vins Padró,NaN,NaN,Straw yellow,Green apple / Citrus fruits / Stone fruit,Flavoursome / Fresh / Persistent,NaN,NaN,NaN,NaN,NaN,Serve between 6ºC and 8ºC,NaN,Contains sulfites,12.5%,NaN,NaN,Mediterranean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ipsis-chard...,2018,NaN
834,Hello World Petit Verdot 2021,10.2,NaN,Red wine,Castilla (Spain),100% Petit Verdot,Finca La Estacada,Owned by the Cantarero Rodríguez family and lo...,NaN,Picota cherry red / Intense / Opaque / Bluish ...,Fruit aromas / Ripe fruit / Liquorice / Eucaly...,Fresh / Intense / Full / Long finish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,2 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/hello-world...,2021,NaN
994,Familia Pacheco Rosado 2022,10.6,NaN,Rosé wine,Jumilla (Spain),100% Monastrell,Bodegas Viña Elena,NaN,NaN,Pale pink / Violet glints / Bright / Crystal-c...,Fruit aromas / Strawberries / Floral scents / ...,Balanced / Good acidity / Lactic notes / Pleas...,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,Contains sulfites,13.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/familia-pac...,2022,NaN
1068,El Jardín de las Iguales Macabeo 2016,160.8,NaN,White wine,Wines without GI (Spain),100% Macabeo,Bodegas Frontonio,Bodegas Frontonio is named after Saint Fronton...,NaN,Yellow / Steely glints,Citrus fruits / Lemon / Clove / Thyme / Fruit ...,Refreshing acidity / Elegant / Vivacious / Spi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.5%,NaN,NaN,120 years,Limestone / Slate,Very low yields,2.00 hectares,NaN,NaN,PARKER,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,97 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/el-jardin-d...,2016,NaN


In [29]:
# Fill NaNs in bottle_x with the value in bottle_y (first in bottle_nulls to check)
bottle_nulls["bottle_x"].fillna(bottle_nulls["bottle_y"], inplace=True)
bottle_nulls.head()

,wine,price,bottle_x,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,bottle_y
88,Viña Tondonia Reserva Blanco 2012,74.2,/ 0.75 L btl,White wine,Rioja (Spain),90% Viura10% Malvasia,Bodegas R. López de Heredia,Tondonia whites are much more than classics. T...,R. López de Heredia (Viña Tondonia) is one of ...,Pale amber,Fine / Complex / Evolving / Crystallised fruit...,Smooth / Firm / Persistent,NaN,NaN,NaN,NaN,NaN,Serve between 14ºC and 16ºC,NaN,Contains sulfites,12.5%,"35,000 bottles",NaN,72 months,Used,American oak,Unfiltered,NaN,NaN,PARKER,100 years,Alluvial / Limestone,Atlantic,110.00 hectares,NaN,4.7/5,36 reviews,95 PK,NaN,2012.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-tondon...,2012,/ 0.75 L btl
776,Ipsis Chardonnay 2018,11.4,NaN,White wine,Tarragona (Spain),100% Chardonnay,Vins Padró,NaN,NaN,Straw yellow,Green apple / Citrus fruits / Stone fruit,Flavoursome / Fresh / Persistent,NaN,NaN,NaN,NaN,NaN,Serve between 6ºC and 8ºC,NaN,Contains sulfites,12.5%,NaN,NaN,Mediterranean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ipsis-chard...,2018,NaN
834,Hello World Petit Verdot 2021,10.2,NaN,Red wine,Castilla (Spain),100% Petit Verdot,Finca La Estacada,Owned by the Cantarero Rodríguez family and lo...,NaN,Picota cherry red / Intense / Opaque / Bluish ...,Fruit aromas / Ripe fruit / Liquorice / Eucaly...,Fresh / Intense / Full / Long finish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,2 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/hello-world...,2021,NaN
994,Familia Pacheco Rosado 2022,10.6,NaN,Rosé wine,Jumilla (Spain),100% Monastrell,Bodegas Viña Elena,NaN,NaN,Pale pink / Violet glints / Bright / Crystal-c...,Fruit aromas / Strawberries / Floral scents / ...,Balanced / Good acidity / Lactic notes / Pleas...,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,Contains sulfites,13.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/familia-pac...,2022,NaN
1068,El Jardín de las Iguales Macabeo 2016,160.8,NaN,White wine,Wines without GI (Spain),100% Macabeo,Bodegas Frontonio,Bodegas Frontonio is named after Saint Fronton...,NaN,Yellow / Steely glints,Citrus fruits / Lemon / Clove / Thyme / Fruit ...,Refreshing acidity / Elegant / Vivacious / Spi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.5%,NaN,NaN,120 years,Limestone / Slate,Very low yields,2.00 hectares,NaN,NaN,PARKER,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,97 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/el-jardin-d...,2016,NaN


In [30]:
# Fill NaNs in bottle_x with the value in bottle_y in the vinissimus dataframe 
vinissimus["bottle_x"].fillna(vinissimus["bottle_y"], inplace=True)
vinissimus["bottle_x"].isna().sum()

563

We still have 556 NaN values in this column, but we have managed to fill more than 1200 NaN values.

In [31]:
# Rename bottle_x and remove bottle_y

vinissimus = vinissimus.rename(columns={"bottle_x":"bottle"}).drop("bottle_y", axis=1)
vinissimus.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year
0,Ultreia Saint Jacques 2021,16.2,/ 0.75 L btl,Red wine,Bierzo (Spain),100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5%,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,NaN,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2/5,217 reviews,93-94 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021
1,Blanc Pescador,7.8,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4/5,45 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>
2,Bach Extrísimo Blanco Semidulce 2021,9.4,/ 0.75 L btl,White wine,Catalunya (Spain),Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5%,NaN,NaN,6 months,American oak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8/5,95 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,2021
3,José Pariente Verdejo 2022,16.2,/ 0.75 L btl,White wine,Rueda (Spain),100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,40.0,30.0,70.0,NaN,"Due to its freshness and acidity, it works ver...",Serve between 6ºC and 8ºC,FishSaladsStarters and suppersVegetablesSoft c...,Contains sulfites,13.0%,NaN,It is produced using grapes from both the fami...,Stainless steel / Concrete / Wood,4 months,NaN,NaN,NaN,"Bodegas José Pariente started up in the 1960s,...",NaN,Stony / Sand,Continental with Atlantic influence,NaN,NaN,NaN,4.4/5,147 reviews,NaN,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,2022
4,Coto de Imaz Reserva 2018,13.0,/ 0.75 L btl,Red wine,Rioja (Spain),100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,70.0,60.0,20.0,60.0,El Coto de Imaz has w

In [32]:
# Remove extra characters in "bottle" and set type as float

vinissimus["bottle"] = vinissimus["bottle"].str.replace(" L btl", "").str.replace(" / ", "").astype(float)
vinissimus["bottle"].value_counts(dropna=False)

0.75    4625
NaN      563
0.50      94
0.37      44
3.00      14
5.00       7
1.50       7
0.35       1
Name: bottle, dtype: int64

### Type of wine
There are too many categories in this column, however, it could be interesting to keep 2 columns with the type of wine: 
- A more general classification (red, white, rosé, sparkling, dessert)
- Another one keeping the details 

In [33]:
vinissimus["type_wine"].value_counts(dropna=False)

Red wine                              2664
White wine                            1410
Cava                                   433
Rosé wine                              242
Red Wine                                90
White Wine                              65
Natural sweet wine                      64
Oak-aged Rosé Wine                      36
Orange wine                             32
Pet Nat wine                            28
Late harvest                            26
Sweet wine                              25
Amontillado sherry                      25
Palo Cortado sherry                     23
Fino sherry                             23
Oloroso sherry                          22
Manzanilla sherry                       21
Raisin wine                             16
Vino rancio                             15
Cream Sherry                            10
Clarete wine                            10
Nouveau wine                            10
Sparkling wine                           7
Semi-sweet 

In [34]:
# Unify the use of capital letters

vinissimus["type_wine"] = vinissimus["type_wine"].str.replace("Wine", "wine").str.replace("Sherry", "sherry")
vinissimus["type_wine"].value_counts()

Red wine                              2754
White wine                            1475
Cava                                   433
Rosé wine                              245
Natural sweet wine                      64
Oak-aged Rosé wine                      36
Orange wine                             32
Pet Nat wine                            28
Late harvest                            26
Sweet wine                              25
Amontillado sherry                      25
Fino sherry                             23
Palo Cortado sherry                     23
Oloroso sherry                          22
Manzanilla sherry                       21
Raisin wine                             16
Vino rancio                             15
Nouveau wine                            10
Clarete wine                            10
Cream sherry                            10
Semi-sweet wine                          7
Sparkling wine                           7
Semi-sparkling sweet wine                7
Noble Rot w

In [35]:
# There are 3 "wines"

vinissimus[vinissimus["type_wine"]=="wine"]

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year
4631,Mas Comtal Blanc (Bag in box 5L),43.85,5.00,wine,Penedès (Spain),100% Chardonnay,Mas Comtal,Albert and Joan Mila decided in the early 1980...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/mas-comtal-...,<NA>
5273,Finca Calvestra Brut Nature 2015,31.75,0.75,wine,Wines without GI (Spain),ChardonnayMerseguera,Bodega Mustiguillo,The Finca Terrerazo estate was acquired by the...,NaN,Straw yellow / Fine bubbles,White flowers / Fennel / Thyme / Dried fruit a...,Good acidity / Fresh / Elegant / Creamy / Pers...,NaN,NaN,NaN,NaN,NaN,Serve between 7ºC and 9ºC,Snacks and appetizersStarters and suppersSausa...,Contains sulfites,12.5%,NaN,NaN,Stainless steel,60 months,NaN,NaN,NaN,NaN,ORGANIC,Silty loam,Mediterranean,900.00 meters,NaN,NaN,4.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/finca-calve...,2015
5325,Planetes de Nin Garnatxes 2021,34.25,NaN,wine,Priorat (Spain),100% Garnacha,Família Nin-Ortiz,"Up until now, not many people have heard of Es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,14.5%,"3,670 bottles",NaN,Amphoras,7 months,NaN,NaN,NaN,NaN,PARKER,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,93+ PK,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/planetes-de...,2021


In [36]:
type_wine_index = vinissimus[vinissimus["type_wine"]=="wine"].index
type_wine_index

Int64Index([4631, 5273, 5325], dtype='int64')

In [37]:
# The first one is a white wine (blanc, 100% chardonnay)

vinissimus.at[type_wine_index[0], "type_wine"] = "White"

# Second is Brut Nature = Sparkling

vinissimus.at[type_wine_index[1], "type_wine"] = "Sparkling"

# Third is a red wine

vinissimus.at[type_wine_index[2], "type_wine"] = "Red"

# Check if it changed correctly

vinissimus.loc[type_wine_index]

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year
4631,Mas Comtal Blanc (Bag in box 5L),43.85,5.00,White,Penedès (Spain),100% Chardonnay,Mas Comtal,Albert and Joan Mila decided in the early 1980...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/mas-comtal-...,<NA>
5273,Finca Calvestra Brut Nature 2015,31.75,0.75,Sparkling,Wines without GI (Spain),ChardonnayMerseguera,Bodega Mustiguillo,The Finca Terrerazo estate was acquired by the...,NaN,Straw yellow / Fine bubbles,White flowers / Fennel / Thyme / Dried fruit a...,Good acidity / Fresh / Elegant / Creamy / Pers...,NaN,NaN,NaN,NaN,NaN,Serve between 7ºC and 9ºC,Snacks and appetizersStarters and suppersSausa...,Contains sulfites,12.5%,NaN,NaN,Stainless steel,60 months,NaN,NaN,NaN,NaN,ORGANIC,Silty loam,Mediterranean,900.00 meters,NaN,NaN,4.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/finca-calve...,2015
5325,Planetes de Nin Garnatxes 2021,34.25,NaN,Red,Priorat (Spain),100% Garnacha,Família Nin-Ortiz,"Up until now, not many people have heard of Es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,14.5%,"3,670 bottles",NaN,Amphoras,7 months,NaN,NaN,NaN,NaN,PARKER,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,93+ PK,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/planetes-de...,2021


In [38]:
# Keep this column as a more detailed info

vinissimus["type_wine_details"] = vinissimus["type_wine"]
keep_cols.append("type_wine_details")

In [39]:
vinissimus["type_wine_details"].value_counts()

Red wine                              2754
White wine                            1475
Cava                                   433
Rosé wine                              245
Natural sweet wine                      64
Oak-aged Rosé wine                      36
Orange wine                             32
Pet Nat wine                            28
Late harvest                            26
Amontillado sherry                      25
Sweet wine                              25
Palo Cortado sherry                     23
Fino sherry                             23
Oloroso sherry                          22
Manzanilla sherry                       21
Raisin wine                             16
Vino rancio                             15
Nouveau wine                            10
Clarete wine                            10
Cream sherry                            10
Semi-sweet wine                          7
Semi-sparkling sweet wine                7
Sparkling wine                           7
Noble Rot w

In [40]:
# Remove the "wine" word since it doesn't add information

vinissimus["type_wine_details"] = vinissimus["type_wine_details"].str.replace(" wine", "")
vinissimus["type_wine_details"].value_counts()

Red                                   2755
White                                 1476
Cava                                   433
Rosé                                   245
Natural sweet                           64
Oak-aged Rosé                           36
Orange                                  32
Pet Nat                                 28
Late harvest                            26
Sweet                                   25
Amontillado sherry                      25
Palo Cortado sherry                     23
Fino sherry                             23
Oloroso sherry                          22
Manzanilla sherry                       21
Raisin                                  16
Vino rancio                             15
Nouveau                                 10
Clarete                                 10
Cream sherry                            10
Sparkling                                8
Semi-sweet                               7
Semi-sparkling sweet                     7
Noble Rot  

In [41]:
# Reduce categories in type_wine, starting with dessert wines

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Dessert" if "sherry" in x else x)

other_dessert = ["Sweet wine", "Natural sweet wine", "Fortified wine", "Icewine", "Mistela", "Vino rancio", 
                 "Late harvest", "Raisin wine", "Noble Rot wine", "Fondillón", "Flavoured sweet wine",
                 "Fortified wine Amontillado", "Vino Naranja del Condado de Huelva", "Semi-sweet wine",
                 "Semi-sweet wine", "Fortified wine Fino"]

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Dessert" if x in other_dessert else x)

vinissimus["type_wine"].value_counts()

Red wine                     2754
White wine                   1475
Cava                          433
Dessert                       305
Rosé wine                     245
Oak-aged Rosé wine             36
Orange wine                    32
Pet Nat wine                   28
Clarete wine                   10
Nouveau wine                   10
Sparkling wine                  7
Semi-sparkling sweet wine       7
Rosé wine Young                 5
Frizzante white wine            4
Frizzante rosé wine             1
White                           1
Sparkling                       1
Red                             1
Name: type_wine, dtype: int64

In [42]:
# Sparkling 

sparkling = ["Cava", "Sparkling wine", "Semi-sparkling sweet wine", "Frizzante rosé wine",
             "Frizzante white wine", "Pet Nat wine"]

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Sparkling" if x in sparkling else x)
vinissimus["type_wine"].value_counts()

Red wine              2754
White wine            1475
Sparkling              481
Dessert                305
Rosé wine              245
Oak-aged Rosé wine      36
Orange wine             32
Nouveau wine            10
Clarete wine            10
Rosé wine Young          5
White                    1
Red                      1
Name: type_wine, dtype: int64

In [43]:
# Rosé

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Rosé" if "Rosé" in x else x)
vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Rosé" if x=="Clarete wine" else x)
vinissimus["type_wine"].value_counts()

Red wine        2754
White wine      1475
Sparkling        481
Dessert          305
Rosé             296
Orange wine       32
Nouveau wine      10
White              1
Red                1
Name: type_wine, dtype: int64

In [44]:
# Red

red = ["Red wine", "Nouveau wine"]

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "Red" if x in red else x)
vinissimus["type_wine"].value_counts()

Red            2765
White wine     1475
Sparkling       481
Dessert         305
Rosé            296
Orange wine      32
White             1
Name: type_wine, dtype: int64

In [45]:
# White

white = ["White wine", "Orange wine"]

vinissimus["type_wine"] = vinissimus["type_wine"].apply(lambda x: "White" if x in white else x)
vinissimus["type_wine"].value_counts()

Red          2765
White        1508
Sparkling     481
Dessert       305
Rosé          296
Name: type_wine, dtype: int64

### Winery

In [46]:
vinissimus["winery"].nunique()

1208

In [47]:
vinissimus[vinissimus["winery"].str.startswith("Bodega")].shape

(1711, 44)

We will create a new column "winery_norm" normalizing the values for the purpose of comparing with other sources of data.

In [48]:
def normalize_wineries(text):
    # Check if the input is NaN, and return NaN directly
    if pd.isna(text):
        return text
    else:
        # Replace punctuation symbols and convert to lower case
        text = re.sub(r'[,.:()\-\']', '', text).lower()
        
        # Remove punctuation symbols and convert to lower case
        text = text.replace(r'[^\w\s]', '').lower()
        
        # Remove the words "bodega" and "bodegas" (winery and wineries)
        text = text.replace("bodegas ", "").replace("bodega ", "")       
        
        # Remove "S.L." and "S.A."
        text = text.replace(" s l", "").replace(" s a", "").replace(" sl", "").replace(" sa", "")    
 
        # Normalize accents using unidecode
        text = unidecode(text)
    
    return text

In [49]:
vinissimus["winery_norm"] = vinissimus["winery"].apply(normalize_wineries)

In [50]:
vinissimus["winery_norm"].value_counts()

perelada             40
pinord               33
codorniu             29
grandes vinos        27
bilbainas            25
                     ..
massard/brunet        1
the wine love         1
reina de castilla     1
celler laurona        1
cellernmarti          1
Name: winery_norm, Length: 1208, dtype: int64

In [51]:
keep_cols.append("winery_norm")

### Region

In [52]:
# Remove the country from the "region" column

vinissimus[["region", "country"]] = vinissimus["region"].str.split("(", expand = True)
vinissimus.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,country
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Bierzo,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5%,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,NaN,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2/5,217 reviews,93-94 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021,Red,raul perez viticultor,Spain)
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4/5,45 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Spain)
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Catalunya,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5%,NaN,NaN,6 months,American oak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8/5,95 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,2021,White,masia bach,Spain)
3,José Pariente Verdejo 2022,16.2,0.75,White,Rueda,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,40.0,30.0,70.0,NaN,"Due to its freshness and acidity, it works ver...",Serve between 6ºC and 8ºC,FishSaladsStarters and suppersVegetablesSoft c...,Contains sulfites,13.0%,NaN,It is produced using grapes from both the fami...,Stainless steel / Concrete / Wood,4 months,NaN,NaN,NaN,"Bodegas José Pariente started up in the 1960s,...",NaN,Stony / Sand,Continental with Atlantic influence,NaN,NaN,NaN,4.4/5,147 reviews,NaN,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,2022,White,jose pariente,Spain)
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvet

In [53]:
vinissimus["country"].value_counts()

Spain)    5355
Name: country, dtype: int64

In [54]:
vinissimus = vinissimus.drop("country", axis=1)
vinissimus.columns

Index(['wine', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'winery',
       'wine_description', 'tasting_descr', 'tasting_view', 'tasting_bouquet',
       'tasting_mouth', 'light_bold_perc', 'smooth_tannic_perc',
       'dry_sweet_perc', 'soft_acidic_perc', 'serving_storage_conditions',
       'serve_temperature', 'food_pairing', 'allergens', 'alcohol_content',
       'production', 'winemaking_info', 'vessel_material', 'aging_period',
       'casks_age', 'casks_wood', 'more_winemaking_info', 'vineyards',
       'type_agriculture', 'vineyard_age', 'soil_type', 'climate',
       'orientation', 'surface', 'customer_reviews', 'customer_reviews_number',
       'last_score_parker', 'last_score_penin', 'last_score_year', 'image',
       'url', 'year', 'type_wine_details', 'winery_norm'],
      dtype='object')

In [55]:
# Remove trailing spaces 

vinissimus["region"] = vinissimus["region"].str.rstrip()

**Categories in region** <br>
As with the type of wine, there are too many categories in this column, therefore we are going to have 2 columns: 
- "Region" with the Autonomous Communities (Comunidades Autónomas, CCAA), there are 15 in Spain (+ 2 autonomous cities that have no wine production).
- Another one keeping the details of all the geographical indications.

Some nice resources about the wine regions in Spain:
- https://www.alimentosdespana.es/images/es/Cartel_mapa_vino_nov20_web_tcm69-569838.pdf <br>
The map shows the 97 existing quality seals in Spain; 68 D.O., 20 V.P., 7 V.C. and 2 D.O.Ca. However, the map does not include the 42 I.G.P. wines of Spain.
- https://www.foodswinesfromspain.com/en/food/do-and-igp-maps#mybook107/ <br>
Map with the I.G.P. wines
- https://www.mapa.gob.es/es/alimentacion/temas/calidad-diferenciada/dop-igp/ <br>
Searcher of DOP and IGP products in Spain
- https://www.aprenderdevino.es/indicaciones-geograficas-vino-espana/ <br>
Article explaining the terms (in Spanish)
- https://www.vinetur.com/2022112471367/cuantas-igps-y-dops-hay-en-espana.html <br>
Article with list of regions (in Spanish)

In [56]:
vinissimus["region"].nunique()

97

In [57]:
vinissimus["region_gi"] = vinissimus["region"]
keep_cols.append("region_gi")

In [58]:
def get_ca(text):
    values = text.splitlines()
    list_ca = [value.split(" ", 1)[1] for value in values]
    return list_ca

In [59]:
# Andalucía

text = '''DO Condado de Huelva
DO Granada
DO Jerez
VC Lebrija
DO Condado de Huelva
DO Granada
DO Jerez
VC Lebrija
DO Málaga
DO Manzanilla
DO Montilla-Moriles
DO Sierras de Málaga
VT Altiplano de Sierra Nevada
VT Bailén
VT Cádiz
VT Córdoba
VT Cumbres del Guadalfeo
VT Desierto de Almería
VT Laderas del Genil
VT Laujar-Alpujarra
VT Los Palacios
VT Norte de Almería
VT Ribera del Andarax
VT Sierra Norte de Sevilla
VT Sierra Sur de Jaén
VT Sierras de Las Estancias y Los Filabres
VT Torreperogil
VT Villaviciosa de Córdoba'''

andalucia = get_ca(text)
andalucia.append("Jerez-Manzanilla")
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Andalucía" if x in andalucia else x)

In [60]:
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Andalucía" if x in andalucia else x)

In [61]:
# Aragón

text = '''VP Aylés
DO Calatayud
DO Campo de Borja
DO Cariñena
DO Somontano
VT Bajo Aragón
VT Ribera del Gállego - Cinco Villas
VT Ribera del Jiloca
VT Valdejalón
VT Valle del Cinca'''

aragon = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Aragón" if x in aragon else x)

In [62]:
# Asturias

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Asturias" if x == "Cangas" else x)

In [63]:
# Islas Baleares

baleares = ["Mallorca", "Pla i Llevant", "Binissalem-Mallorca", "Ibiza", "Formentera", "Illa de Menorca"]

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Islas Baleares" if x in baleares else x)

In [64]:
# Islas Canarias

canarias = ["Abona", "El Hierro", "Gran Canaria", "Islas Canarias", "La Gomera", "La Palma", "Lanzarote",
            "Tacoronte - Acentejo", "Valle de Güímar", "Valle de La Orotava", "Ycoden-Daute-Isora",
            "Canary Islands"]

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Islas Canarias" if x in canarias else x)

In [65]:
# Cantabria

text = '''VT Costa de Cantabria
VT Liébana'''

cantabria = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Cantabria" if x in cantabria else x)

In [66]:
# Castilla y León

text = '''DO Abadía Retuerta
DO Arlanza
DO Arribes
DO Bierzo
VC Cebreros
DO Cigales
DO Dehesa Peñalba
DO León
DO Ribera del Duero
DO Rueda
VC Sierra de Salamanca
DO Tierra del Vino de Zamora
DO Toro
DO Urueña
VC Valles de Benavente
VC Valtiendas
VT Castilla y León'''

cyl = get_ca(text)
cyl.append("Castile and León")
cyl.append("Tierra de León")
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Castilla y León" if x in cyl else x)

In [67]:
# Castilla-La Mancha

text = '''DO Almansa
VP Calzadilla
VP Campo de La Guardia
VP Casa del Blanco
VP Dehesa del Carrizal
VP Dominio de Valdepusa
VP El Vicario
VP Finca Élez
VP Guijoso
VP La Jaraba
DO La Mancha
VP Los Cerrillos
DO Manchuela
DO Méntrida
DO Mondéjar
VP Pago Florentino
DO Ribera del Júcar
DO Uclés
DO Valdepeñas
VP Vallegarcía
VT Castilla
'''

clm = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Castilla-La Mancha" if x in clm else x)

In [68]:
# Cataluña 

text = '''DO Alella
DO Catalunya
DO Conca de Barberà
DO Costers del Segre
DO Empordà
DO Montsant
DO Penedès
DO Pla de Bages
DOCa Priorat
DO Tarragona
DO Terra Alta
'''

cataluna = get_ca(text)
more_cat = ["Corpinnat", "Carignan", "Clàssic Penedès", "Conca del Riu Anoia"]
cataluna = cataluna + more_cat
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Cataluña" if x in cataluna else x)

In [69]:
# Galicia 

text = '''DO Monterrei
DO Rías Baixas
DO Ribeira Sacra
DO Ribeiro
DO Valdeorras
VT Barbanza e Iria
VT Betanzos
VT Ribeiras do Morrazo
VT Val do Miño-Ourense
'''

galicia = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Galicia" if x in galicia else x)

In [70]:
# Extremadura 

text = '''DO Ribera del Guadiana
VT Extremadura
'''

extremadura = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Extremadura" if x in extremadura else x)

In [71]:
# La Rioja

vinissimus["region"] = vinissimus["region"].apply(lambda x: "La Rioja" if x == "Valles de Sadacia" else x)

In [72]:
# Madrid

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Madrid" if x == "Vinos de Madrid" else x)

In [73]:
# Región de Murcia 

text = '''DO Bullas
DO Yecla
VT Campo de Cartagena
VT Murcia
'''

murcia = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Región de Murcia" if x in murcia else x)

In [74]:
# Navarra 

text = '''DO Bolandin
DO Navarra
VP Pago de Arínzano
VP Pago de Otazu
VP Prado de Irache
VT 3 Riberas
'''

navarra = get_ca(text)
navarra.append("Navarre")
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Navarra" if x in navarra else x)

In [75]:
# Euskadi

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Euskadi" if "Txakoli" in x else x)

In [76]:
# Comunidad Valenciana

text = '''DO Alicante
VP Chozas Carrascal
VP El Terrerazo
VP Los Balagueses
DO Utiel-Requena
DO Valencia
VP Vera de Estenas
VT Castelló
'''

valencia = get_ca(text)
vinissimus["region"] = vinissimus["region"].apply(lambda x: "Comunidad Valenciana" if x in valencia else x)

In [77]:
vinissimus["region"].value_counts()

Cataluña                    1476
Castilla y León              939
Rioja                        684
Galicia                      393
Wines without GI             363
DO Cava                      319
Andalucía                    260
Aragón                       163
Castilla-La Mancha           148
Comunidad Valenciana         102
Navarra                       95
Islas Canarias                95
Islas Baleares                94
Jumilla                       77
Madrid                        49
Región de Murcia              32
Extremadura                   20
Euskadi                       19
La Rioja                       8
Ribera del Queiles             5
Asturias                       4
Pago Aylés                     3
Pago Dehesa del Carrizal       2
Pago Calzadilla                2
Pago Casa del Blanco           2
Pago Campo de la Guardia       1
Name: region, dtype: int64

In [78]:
# Vinos de Pago

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Vino de Pago" if "Pago" in x else x)

In [79]:
vinissimus["region"].nunique()

22

We have every autonomous communities in Spain represented in the dataset, except one (Cantabria). We also have several  "supraautonomical" categories:
- Wines without GI
- Vino de Pago
- DO Cava (Cataluña, La Rioja, Álava (Euskadi), Zaragoza (Aragón), Navarra, Valencia and Extremadura)
- DO Rioja (La Rioja, Álava (Euskadi))
- DO Jumilla (Murcia, Albacete (Castilla-La Mancha))
- VT Ribera del Queiles (Aragón and Navarra)

#### Manually add some wines from Cantabria 
There are two GI in Cantabria (Costa de Cantabria and Liébana), we will add one wine of each GI (for the second one we had to combine two sources).
- https://vinopremier.com/vino-blanco-vi-a-mar.html 
- https://casacano.es/tinto/148-vino-tinto-lusia-do-liebana-75cl.html
- https://www.elcorteingles.es/supermercado/0110118747001057-lusia-vino-tinto-roble-vino-de-la-tierra-de-liebana-botella-75-cl/

In [80]:
new_row = {'wine': "Viña Mar",
           'year': 2019,
           'winery': "Bodegas Pago Casa del Blanco", 
           'price': 11,
           'bottle': 0.75,
           'type_wine': "White",
           'type_wine_details': "White",
           'grapes': "Albariño",
           'type_agriculture': "Conventional",
           'region': "Cantabria",
           'region_gi': "Costa de Cantabria",
           'alcohol_content': 12,
           'tasting_notes': "white_fruit stone_fruit floral fresh citrus",
           'image': "https://vinopremier.com/media/catalog/product/cache/070e1948fa79794c1092c01c58a07334/v/i/vino-blanco-vi_amar-vinopremier.jpg",
           'url': "https://vinopremier.com/vino-blanco-vi-a-mar.html"}

vinissimus = vinissimus.append(new_row, ignore_index=True)

In [81]:
new_row = {'wine': "Lusia 2020",
           'year': 2020,
           'winery': "Casa Cano", 
           'price': 16.5,
           'bottle': 0.75,
           'type_wine': "Red",
           'type_wine_details': "Red",
           'grapes': "MencíaTempranillo",
           'type_agriculture': "Conventional",
           'region': "Cantabria",
           'region_gi': "Liébana",
           'alcohol_content': 12.5,
           'tasting_notes': "red_fruit smoke wood astringent",
           'image': "https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA03/202005/31/00118747001057____1__1200x1200.jpg",
           'url': "https://www.elcorteingles.es/supermercado/0110118747001057-lusia-vino-tinto-roble-vino-de-la-tierra-de-liebana-botella-75-cl/"}

vinissimus = vinissimus.append(new_row, ignore_index=True)

In [82]:
vinissimus.tail()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes
5352,Tianna Vélorosé 2020,17.45,0.75,Rosé,Islas Baleares,100% Mantonegro,Celler Tianna Negre,NaN,NaN,Salmon / Pale,Delicate / Fruit aromas / Pomegranate / Vine p...,Full / Rich / Fruit-forward / Oily,NaN,NaN,NaN,NaN,NaN,Serve between 6ºC and 7ºC,NaN,Contains sulfites,13.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ORGANIC,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/tianna-velo...,2020,Rosé,celler tianna negre,Binissalem-Mallorca,NaN
5353,Gran Campo Viejo Cava Brut Reserva,12.40,NaN,Sparkling,DO Cava,ParelladaMacabeoXarel·lo,Bodegas Campo Viejo,Campo Viejo is one of the Rioja region's most ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4/5,5 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/gran-campo-...,<NA>,Cava,campo viejo,DO Cava,NaN
5354,Manyol Selecció 2019,12.40,0.75,Red,Cataluña,GarnachaSyrah,Viticultors Bateans,Some of the best wines from Terra Alta leave f...,NaN,Picota cherry red / Violet glints / Clean / Br...,Fruit aromas / Fruits of the forest / Jam / Bl...,Structured / Full / Opulent / Velvety / Spicy ...,NaN,NaN,NaN,NaN,NaN,Serve between 13ºC and 15ºC,NaN,Contains sulfites,14.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/manyol-sele...,2019,Red,viticultors bateans,Terra Alta,NaN
5355,Viña Mar,11.00,0.75,White,Cantabria,Albariño,Bodegas Pago Casa del Blanco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://vinopremier.com/media/catalog/product/...,https://vinopremier.com/vino-blanco-vi-a-mar.html,2019,White,NaN,Costa de Cantabria,white_fruit stone_fruit floral fresh citrus
5356,Lusia 2020,16.50,0.75,Red,Cantabria,MencíaTempranillo,Casa Cano,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,https://www.elcorteingles.es/supermercado/0110...,2020,Red,NaN,Liébana,red_fruit smoke wood astringent


#### Supraautonomical categories
We will try to assign each wine of the "supraautonomical categories" to their CCAA, at least in the most important GIs. For that purpose, we have webscraped the official webpages of the GIs to obtain the list of wineries. We use those lists to assign the wineries to their corresponding CCAA. 

In [83]:
supra = pd.read_csv("../01-initial_data/webscraping/supraautonomical_gis.csv")
supra.head()

,rioja_alavesa,cava_cataluña,cava_extremadura,cava_valencia,jumilla_murcia,jumilla_albacete
0,Amador García,ADERNATS - Vinícola de Nulles,Bodegas Dehesa de Arriba (Grupo Martínez Paiva),BODEGAS HISPANOSUIZAS S.L.,Bodega El Lagar de las Puntillas,Bodega Antonio García Molina
1,Dominio de Berzal,AGRÍCOLA I SECCIÓ DE CRÈDIT DE LLORENÇ DEL PEN...,BODEGAS MARCELINO DIAZ S.A.,Chozas Carrascal,Bodega Madrid Romero,Bodega Cerrón
2,Luberri Monje Amestoy,AGUSTÍ TORELLÓ MATA,Bodegas Romale,DOMINIO DE LA VEGA,Bodega Sierra Norte,Bodegas Ontalba
3,Murua,ALTA ALELLA,López Morenas S.L.,MONTESANCO,Bodegas 1890,Bodegas Pío del Ramo
4,Pago de Larrea,ARTCAVA,"VIA DE LA PLATA, S. L.",PAGO DE THARSYS,Bodegas Alceño,Bodegas San Dionisio


In [84]:
# Normalize the values in the dataframe

for col in supra.columns:
    supra[col] = supra[col].apply(normalize_wineries)
    
supra.head()

,rioja_alavesa,cava_cataluña,cava_extremadura,cava_valencia,jumilla_murcia,jumilla_albacete
0,amador garcia,adernats vinicola de nulles,dehesa de arriba grupo martinez paiva,hispanosuizas,el lagar de las puntillas,antonio garcia molina
1,dominio de berzal,agricola i seccio de credit de llorenc del pen...,marcelino diaz,chozas carrascal,madrid romero,cerron
2,luberri monje amestoy,agusti torello mata,romale,dominio de la vega,sierra norte,ontalba
3,murua,alta alella,lopez morenas,montesanco,1890,pio del ramo
4,pago de larrea,artcava,via de la plata,pago de tharsys,alceno,san dionisio


In [85]:
def assign_region(df, col, region, region_gi, supra):
    col_supra = supra[[col]]
    col_supra = col_supra.dropna()
    col_list = list(col_supra[col])
    
     # Create a boolean mask for the rows where the condition is met
    mask = df["winery_norm"].isin(col_list) & (df["region_gi"] == region_gi)

    # Update the "region" column only for the rows where the condition is True
    df.loc[mask, "region"] = region

    return df

In [86]:
vinissimus["region"].value_counts()

Cataluña                1476
Castilla y León          939
Rioja                    684
Galicia                  393
Wines without GI         363
DO Cava                  319
Andalucía                260
Aragón                   163
Castilla-La Mancha       148
Comunidad Valenciana     102
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Jumilla                   77
Madrid                    49
Región de Murcia          32
Extremadura               20
Euskadi                   19
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [87]:
vinissimus = assign_region(vinissimus, "rioja_alavesa", "Euskadi", "Rioja", supra)
vinissimus["region"].value_counts()

Cataluña                1476
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
DO Cava                  319
Andalucía                260
Aragón                   163
Castilla-La Mancha       148
Euskadi                  111
Comunidad Valenciana     102
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Jumilla                   77
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [88]:
vinissimus = assign_region(vinissimus, "cava_cataluña", "Cataluña", "DO Cava", supra)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  223
Aragón                   163
Castilla-La Mancha       148
Euskadi                  111
Comunidad Valenciana     102
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Jumilla                   77
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [89]:
vinissimus = assign_region(vinissimus, "cava_extremadura", "Extremadura", "DO Cava", supra)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  223
Aragón                   163
Castilla-La Mancha       148
Euskadi                  111
Comunidad Valenciana     102
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Jumilla                   77
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [90]:
vinissimus = assign_region(vinissimus, "cava_valencia", "Comunidad Valenciana", "DO Cava", supra)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  221
Aragón                   163
Castilla-La Mancha       148
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Jumilla                   77
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [91]:
vinissimus = assign_region(vinissimus, "jumilla_murcia", "Murcia", "Jumilla", supra)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  221
Aragón                   163
Castilla-La Mancha       148
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Murcia                    51
Madrid                    49
Región de Murcia          32
Jumilla                   26
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [92]:
vinissimus = assign_region(vinissimus, "jumilla_albacete", "Castilla-La Mancha", "Jumilla", supra)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
Rioja                    592
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  221
Aragón                   163
Castilla-La Mancha       151
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Murcia                    51
Madrid                    49
Región de Murcia          32
Jumilla                   23
Extremadura               20
Vino de Pago              10
La Rioja                   8
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [93]:
# Assign the rest of the Rioja wines to the CCAA La Rioja

vinissimus["region"] = vinissimus["region"].apply(lambda x: "La Rioja" if x == "Rioja" else x)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
La Rioja                 600
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  221
Aragón                   163
Castilla-La Mancha       151
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Murcia                    51
Madrid                    49
Región de Murcia          32
Jumilla                   23
Extremadura               20
Vino de Pago              10
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [94]:
# Assign the rest of the Jumilla wines to the CCAA Murcia

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Murcia" if x == "Jumilla" else x)
vinissimus["region"].value_counts()

Cataluña                1572
Castilla y León          939
La Rioja                 600
Galicia                  393
Wines without GI         363
Andalucía                260
DO Cava                  221
Aragón                   163
Castilla-La Mancha       151
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Murcia                    74
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [95]:
# Assign the rest of the Cava wines to the CCAA Cataluña

vinissimus["region"] = vinissimus["region"].apply(lambda x: "Cataluña" if x == "DO Cava" else x)
vinissimus["region"].value_counts()

Cataluña                1793
Castilla y León          939
La Rioja                 600
Galicia                  393
Wines without GI         363
Andalucía                260
Aragón                   163
Castilla-La Mancha       151
Euskadi                  111
Comunidad Valenciana     104
Navarra                   95
Islas Canarias            95
Islas Baleares            94
Murcia                    74
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
Ribera del Queiles         5
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

In [96]:
# Ribera del Queiles: manually assign this 5 wines
vinissimus[vinissimus["region"]=="Ribera del Queiles"]

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes
2348,Erial 2020,16.80,0.75,Red,Ribera del Queiles,100% Tempranillo,Bodegas Epifanio Rivera,NaN,NaN,Violet red / Opaque,Complex / Black fruit / Red fruit / Ageing not...,Lively / Balanced / Elegant,NaN,NaN,NaN,NaN,NaN,Serve between 14ºC and 16ºC,NaN,Contains sulfites,14.5%,"30,000 - 40,000 bottles",NaN,12 months,French and American oak,NaN,NaN,NaN,NaN,NaN,Between 20 and 70 years,Calcareous-clay,Continental,Southwest,NaN,4.5/5,4 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/erial/,2020,Red,epifanio rivera,Ribera del Queiles,NaN
3902,Número Nueve 2017,15.35,0.75,Red,Ribera del Queiles,TempranilloCabernet Franc,Winery Arts,Number 9 is considered by various cultures to ...,Winery Arts makes its debut in the wine world ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve between 16ºC and 18ºC,NaN,Contains sulfites,15.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4/5,5 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/numero-nueve/,2017,Red,winery arts,Ribera del Queiles,NaN
4645,Guelbenzu Vierlas 2017,11.95,0.75,Red,Ribera del Queiles,33% Syrah23% Cabernet Sauvignon17% Graciano13%...,Bodegas Guelbenzu,An original blend of varieties from the vineya...,Bodegas Guelbenzu looks back over a long histo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0/5,2 reviews,90 PK,NaN,2006.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/guelbenzu-v...,2017,Red,guelbenzu,Ribera del Queiles,NaN
4706,Guelbenzu Evo 2015,20.10,NaN,Red,Ribera del Queiles,60% Cabernet Sauvignon25% Merlot15% Tempranillo,Bodegas Guelbenzu,"A modern-style, straight-forward wine that sho...",Bodegas Guelbenzu looks back over a long histo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,14.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,3 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/guelbenzu-evo/,2015,Red,guelbenzu,Ribera del Queiles,NaN
5034,Come Pollo y Bebe Vino 2017,10.55,0.75,Red,Ribera del Queiles,100% Garnacha,Castillo de Maetierra,Castillo de Maetierra is the only winery in La...,NaN,Picota cherry red / Garnet / Bright / Semi-opaque,Fruit aromas / Wild fruits / Raspberry / Candy...,Silky / Fresh / Mineral notes / Medium-bodied ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contains sulfites,13.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/come-pollo-...,2017,Red,castillo de maetierra,Ribera del Queiles,NaN


In [97]:
# The first one seems to be an error since it is labeled as Ribera del Duero
vinissimus.loc[vinissimus["wine"] == "Erial 2020", "region"] = "Castilla y León"
vinissimus.loc[vinissimus["wine"] == "Erial 2020", "region_gi"] = "Ribera del Duero"

# Guelbenzu is in Zaragoza
vinissimus.loc[vinissimus["wine"] == "Guelbenzu Vierlas 2017", "region"] = "Aragón"
vinissimus.loc[vinissimus["wine"] == "Guelbenzu Evo 2015", "region"] = "Aragón"

# The rest in Navarra
vinissimus.loc[vinissimus["region"] == "Ribera del Queiles", "region"] = "Navarra"

vinissimus["region"].value_counts()

Cataluña                1793
Castilla y León          940
La Rioja                 600
Galicia                  393
Wines without GI         363
Andalucía                260
Aragón                   165
Castilla-La Mancha       151
Euskadi                  111
Comunidad Valenciana     104
Navarra                   97
Islas Canarias            95
Islas Baleares            94
Murcia                    74
Madrid                    49
Región de Murcia          32
Extremadura               20
Vino de Pago              10
Asturias                   4
Cantabria                  2
Name: region, dtype: int64

### Serve temperature

In [98]:
vinissimus["serve_temperature"].nunique()

83

In [99]:
vinissimus["serve_temperature"].value_counts()

Serve between 16ºC and 18ºC    705
Serve between 14ºC and 16ºC    447
Serve between 8ºC and 10ºC     382
Serve between 15ºC and 17ºC    327
Serve between 6ºC and 8ºC      323
                              ... 
Serve between 5ºC and 12ºC       1
Serve between 80ºC and 10ºC      1
Serve between 19ºC and 20ºC      1
Serve between 16ºC and 16ºC      1
Serve between 14ºC and 18ºC      1
Name: serve_temperature, Length: 83, dtype: int64

In [100]:
vinissimus_short = vinissimus[keep_cols]
vinissimus_short.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,allergens,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year,year,type_wine_details,winery_norm,region_gi
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,Contains sulfites,13.5%,4.2/5,217 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,NaN,93-94 PK,NaN,2020.0,2021,Red,raul perez viticultor,Bierzo
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,Contains sulfites,11.5%,4.4/5,45 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,NaN,NaN,NaN,NaN,<NA>,Frizzante white,perelada,Wines without GI
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,Contains sulfites,11.5%,4.8/5,95 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,NaN,NaN,NaN,NaN,2021,White,masia bach,Catalunya
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,Contains sulfites,13.0%,4.4/5,147 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,NaN,NaN,NaN,2021.0,2022,White,jose pariente,Rueda
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,Contains sulfites,13.5%,4.4/5,94 reviews,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,SUCKLING,NaN,NaN,2018.0,2018,Red,el coto de rioja,Rioja


### Allergens

In [101]:
# Some values from "alcohol_content" are in the "allergens" column

vinissimus["allergens"].value_counts(dropna=False)

Contains sulfites    5335
12.5%                   7
13.0%                   5
13.5%                   3
11.5%                   2
12.0%                   2
NaN                     2
11.0%                   1
Name: allergens, dtype: int64

In [102]:
# Assign the values from "allergens" to "alcohol_content" for rows where the "allergens" value is not equal to "Contains sulfites"

vinissimus.loc[vinissimus["allergens"] != "Contains sulfites" , "alcohol_content"] = vinissimus["allergens"]

In [103]:
# This was probably because those wines had a missing element in the webscraping, let's have a look

vinissimus["allergens"] = vinissimus["allergens"].apply(lambda x: np.nan if x != "Contains sulfites" else x)
allergens_nans = vinissimus[vinissimus["allergens"].isna()]
allergens_nans

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes
327,Marqués de Vizhoja 2022,10.40,0.75,White,Wines without GI,Marqués de Vizhoja,Contains sulfites,NaN,NaN,Straw yellow / Greenish glints,Fruit aromas / White fruit / Apple / Pear / Fl...,Fine / Balanced / Citrus notes / Fruit notes,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,11 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/marques-de-...,2022,White,contains sulfites,Wines without GI,NaN
751,Habla del Mar,22.00,0.75,White,Wines without GI,Bodegas Habla,Contains sulfites,Bodegas Habla is a newly-founded winery in Ext...,NaN,Lemony yellow / Clean / Bright / Steely glints,Subtle / Saline notes / Vegetal notes / Algae ...,Smooth / Expressive / Good acidity / Mineral n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,4 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/habla-del-mar/,<NA>,White,contains sulfites,Wines without GI,NaN
870,Castellum Augusti 2019,19.00,0.75,White,Galicia,Bodegas Castellun Augusti,Contains sulfites,NaN,NaN,Deep yellow / Bright / Crystal-clear,Higly aromatic / Fruit aromas / Stone fruit / ...,Balanced / Flavoursome / Unctuous / Rounded / ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,Stainless steel,9 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/castellum-a...,2019,White,contains sulfites,Barbanza e Iria,NaN
1154,Come Pulpo y Bebe Vino 2020,10.80,0.75,White,Wines without GI,Castillo de Maetierra,Contains sulfites,Castillo de Maetierra is the only winery in La...,NaN,Straw yellow / Greenish glints / Golden glints,Fresh / Fruit aromas / Acidic fruit / Apple / ...,Fresh / Slightly acidic / Easy to drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7/5,6 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/come-pulpo-...,2020,White,contains sulfites,Wines without GI,NaN
1356,Vins Padró Blanc (Bag in box 5L),39.40,5.00,White,Wines without GI,Vins Padró,Contains sulfites,NaN,NaN,Pale yellow,White fruit / Fruity,Smooth / Fresh,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vins-padro-...,<NA>,White,contains sulfites,Wines without GI,NaN
1493,Viña Real Bakeder 2019,22.40,0.75,Red,La Rioja,Viña Real,Contains sulfites,The Viña Real brand was created in 1960 by Cvn...,NaN,Picota cherry red / Opaque / Violet glints,Red fruit / Black fruit / Liquorice / Hardwood...,Structured / Corpulent / Fine tannins / Long /...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5%,NaN,NaN,11 months,French oak,NaN,NaN,NaN,NaN,PEÑÍN,30 years,Calcareous-clay,South,8.78 hectares,575.00 meters,0.0/5,0 reviews,92+ PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-real-b...,2019,Red,contains sulfites,Rioja,NaN
1713,Akilia Lombano 2011,30.60,0.75,Red,Castilla y León,Akilia Wines,Contains sulfites,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve between 16ºC and 18ºC,N

In [104]:
# Those wines had no "grapes" value, now we have to fix the "grapes" and the "producer" columns
# Try first with allergens_nans to make sure that the code works

allergens_nans.loc[allergens_nans["winery"] == "Contains sulfites", "temp_col"] = "yes" # creates temporary column to identify the rows
allergens_nans.loc[allergens_nans["winery"] == "Contains sulfites", "winery"] = allergens_nans.loc[allergens_nans["winery"] == "Contains sulfites", "grapes"] # replaces the "producer" info with the "grapes" info
allergens_nans.loc[allergens_nans["temp_col"] == "yes", "grapes"] = np.nan # replaces the "grapes" info with NaNs

allergens_nans

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes,temp_col
327,Marqués de Vizhoja 2022,10.40,0.75,White,Wines without GI,NaN,Marqués de Vizhoja,NaN,NaN,Straw yellow / Greenish glints,Fruit aromas / White fruit / Apple / Pear / Fl...,Fine / Balanced / Citrus notes / Fruit notes,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,11 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/marques-de-...,2022,White,contains sulfites,Wines without GI,NaN,yes
751,Habla del Mar,22.00,0.75,White,Wines without GI,NaN,Bodegas Habla,Bodegas Habla is a newly-founded winery in Ext...,NaN,Lemony yellow / Clean / Bright / Steely glints,Subtle / Saline notes / Vegetal notes / Algae ...,Smooth / Expressive / Good acidity / Mineral n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,4 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/habla-del-mar/,<NA>,White,contains sulfites,Wines without GI,NaN,yes
870,Castellum Augusti 2019,19.00,0.75,White,Galicia,NaN,Bodegas Castellun Augusti,NaN,NaN,Deep yellow / Bright / Crystal-clear,Higly aromatic / Fruit aromas / Stone fruit / ...,Balanced / Flavoursome / Unctuous / Rounded / ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,Stainless steel,9 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/castellum-a...,2019,White,contains sulfites,Barbanza e Iria,NaN,yes
1154,Come Pulpo y Bebe Vino 2020,10.80,0.75,White,Wines without GI,NaN,Castillo de Maetierra,Castillo de Maetierra is the only winery in La...,NaN,Straw yellow / Greenish glints / Golden glints,Fresh / Fruit aromas / Acidic fruit / Apple / ...,Fresh / Slightly acidic / Easy to drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7/5,6 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/come-pulpo-...,2020,White,contains sulfites,Wines without GI,NaN,yes
1356,Vins Padró Blanc (Bag in box 5L),39.40,5.00,White,Wines without GI,NaN,Vins Padró,NaN,NaN,Pale yellow,White fruit / Fruity,Smooth / Fresh,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vins-padro-...,<NA>,White,contains sulfites,Wines without GI,NaN,yes
1493,Viña Real Bakeder 2019,22.40,0.75,Red,La Rioja,NaN,Viña Real,The Viña Real brand was created in 1960 by Cvn...,NaN,Picota cherry red / Opaque / Violet glints,Red fruit / Black fruit / Liquorice / Hardwood...,Structured / Corpulent / Fine tannins / Long /...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5%,NaN,NaN,11 months,French oak,NaN,NaN,NaN,NaN,PEÑÍN,30 years,Calcareous-clay,South,8.78 hectares,575.00 meters,0.0/5,0 reviews,92+ PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-real-b...,2019,Red,contains sulfites,Rioja,NaN,yes
1713,Akilia Lombano 2011,30.60,0.75,Red,Castilla y León,NaN,Akilia Wines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve between 16ºC and 18ºC,NaN,NaN,13.0%,NaN,NaN,Concrete / Wood,12 months,French oak,NaN,NaN

In [105]:
allergens_nans_index = allergens_nans.index

In [106]:
# Now apply to the full dataframe

vinissimus.loc[vinissimus["winery"] == "Contains sulfites", "temp_col"] = "yes"
vinissimus.loc[vinissimus["winery"] == "Contains sulfites", "winery"] = vinissimus.loc[vinissimus["winery"] == "Contains sulfites", "grapes"]
vinissimus.loc[vinissimus["temp_col"] == "yes", "grapes"] = np.nan

vinissimus.loc[allergens_nans_index]

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes,temp_col
327,Marqués de Vizhoja 2022,10.40,0.75,White,Wines without GI,NaN,Marqués de Vizhoja,NaN,NaN,Straw yellow / Greenish glints,Fruit aromas / White fruit / Apple / Pear / Fl...,Fine / Balanced / Citrus notes / Fruit notes,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,11 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/marques-de-...,2022,White,contains sulfites,Wines without GI,NaN,yes
751,Habla del Mar,22.00,0.75,White,Wines without GI,NaN,Bodegas Habla,Bodegas Habla is a newly-founded winery in Ext...,NaN,Lemony yellow / Clean / Bright / Steely glints,Subtle / Saline notes / Vegetal notes / Algae ...,Smooth / Expressive / Good acidity / Mineral n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3/5,4 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/habla-del-mar/,<NA>,White,contains sulfites,Wines without GI,NaN,yes
870,Castellum Augusti 2019,19.00,0.75,White,Galicia,NaN,Bodegas Castellun Augusti,NaN,NaN,Deep yellow / Bright / Crystal-clear,Higly aromatic / Fruit aromas / Stone fruit / ...,Balanced / Flavoursome / Unctuous / Rounded / ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,Stainless steel,9 months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0/5,0 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/castellum-a...,2019,White,contains sulfites,Barbanza e Iria,NaN,yes
1154,Come Pulpo y Bebe Vino 2020,10.80,0.75,White,Wines without GI,NaN,Castillo de Maetierra,Castillo de Maetierra is the only winery in La...,NaN,Straw yellow / Greenish glints / Golden glints,Fresh / Fruit aromas / Acidic fruit / Apple / ...,Fresh / Slightly acidic / Easy to drink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7/5,6 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/come-pulpo-...,2020,White,contains sulfites,Wines without GI,NaN,yes
1356,Vins Padró Blanc (Bag in box 5L),39.40,5.00,White,Wines without GI,NaN,Vins Padró,NaN,NaN,Pale yellow,White fruit / Fruity,Smooth / Fresh,NaN,NaN,NaN,NaN,NaN,Serve between 8ºC and 10ºC,NaN,NaN,11.0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0/5,1 review,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vins-padro-...,<NA>,White,contains sulfites,Wines without GI,NaN,yes
1493,Viña Real Bakeder 2019,22.40,0.75,Red,La Rioja,NaN,Viña Real,The Viña Real brand was created in 1960 by Cvn...,NaN,Picota cherry red / Opaque / Violet glints,Red fruit / Black fruit / Liquorice / Hardwood...,Structured / Corpulent / Fine tannins / Long /...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.5%,NaN,NaN,11 months,French oak,NaN,NaN,NaN,NaN,PEÑÍN,30 years,Calcareous-clay,South,8.78 hectares,575.00 meters,0.0/5,0 reviews,92+ PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-real-b...,2019,Red,contains sulfites,Rioja,NaN,yes
1713,Akilia Lombano 2011,30.60,0.75,Red,Castilla y León,NaN,Akilia Wines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Serve between 16ºC and 18ºC,NaN,NaN,13.0%,NaN,NaN,Concrete / Wood,12 months,French oak,NaN,NaN

In [107]:
vinissimus = vinissimus.drop("temp_col", axis=1) # Drop the temporary column

In [108]:
# Remove the allergens column since all wines contained sulfites

keep_cols.remove("allergens")

### Alcohol content

In [109]:
# Remove extra characters in "alcohol_content"

vinissimus["alcohol_content"].value_counts(dropna=False)

13.5%              894
14.0%              873
14.5%              869
13.0%              754
12.5%              482
12.0%              416
15.0%              368
11.5%              312
15.5%               63
11.0%               59
16.0%               33
17.0%               31
NaN                 22
10.5%               21
18.0%               20
20.0%               20
10.0%               15
17.5%               12
22.0%               11
19.0%               11
21.0%                8
9.5%                 8
5.5%                 6
19.5%                5
21.5%                5
16.5%                4
5.0%                 3
9.0%                 3
7.0%                 3
6.5%                 2
18.5%                2
20.5%                2
14.7%                2
14.9%                1
11.3%                1
10.8%                1
10.6%                1
11.7%                1
10,000  bottles      1
12.9%                1
7.5%                 1
13.7%                1
14.2%                1
12.8%      

In [110]:
# Replace NaNs with "None" to be able to use replace

vinissimus["alcohol_content"] = vinissimus["alcohol_content"].fillna("None") 

In [111]:
vinissimus["alcohol_content"] = vinissimus["alcohol_content"].str.replace("%", "")
vinissimus["alcohol_content"].value_counts()

13.5               894
14.0               873
14.5               869
13.0               754
12.5               482
12.0               416
15.0               368
11.5               312
15.5                63
11.0                59
16.0                33
17.0                31
None                22
10.5                21
18.0                20
20.0                20
10.0                15
17.5                12
22.0                11
19.0                11
21.0                 8
9.5                  8
5.5                  6
19.5                 5
21.5                 5
16.5                 4
5.0                  3
9.0                  3
7.0                  3
6.5                  2
18.5                 2
20.5                 2
14.7                 2
14.9                 1
11.3                 1
10.8                 1
10.6                 1
11.7                 1
10,000  bottles      1
12.9                 1
7.5                  1
13.7                 1
14.2                 1
12.8       

In [112]:
# Replace "None" and other values with NaN and convert the rest to float type

vinissimus["alcohol_content"] = vinissimus["alcohol_content"].apply(lambda x: np.nan if "bottles" in x or x == "None" else float(x))
vinissimus["alcohol_content"].value_counts(dropna=False)

13.5    894
14.0    873
14.5    869
13.0    754
12.5    482
12.0    416
15.0    368
11.5    312
15.5     63
11.0     59
16.0     33
17.0     31
NaN      23
10.5     21
18.0     20
20.0     20
10.0     15
17.5     12
22.0     11
19.0     11
21.0      8
9.5       8
5.5       6
19.5      5
21.5      5
16.5      4
5.0       3
9.0       3
7.0       3
6.5       2
18.5      2
20.5      2
14.7      2
14.9      1
14.2      1
10.8      1
10.6      1
11.7      1
12.9      1
7.5       1
13.7      1
12.8      1
11.3      1
12.7      1
14.4      1
14.8      1
4.5       1
16.2      1
8.0       1
29.0      1
Name: alcohol_content, dtype: int64

### Customer reviews

In [113]:
# Remove extra characters in "customer_reviews" and convert to float type

vinissimus["customer_reviews"] = vinissimus["customer_reviews"].apply(lambda x: str(x)[:-2] if pd.notna(x) else x).astype(float)
vinissimus.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,NaN,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2,217 reviews,93-94 PK,NaN,2020.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021,Red,raul perez viticultor,Bierzo,NaN
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4,45 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Wines without GI,NaN
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5,NaN,NaN,6 months,American oak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8,95 reviews,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,2021,White,masia bach,Catalunya,NaN
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,40.0,30.0,70.0,NaN,"Due to its freshness and acidity, it works ver...",Serve between 6ºC and 8ºC,FishSaladsStarters and suppersVegetablesSoft c...,Contains sulfites,13.0,NaN,It is produced using grapes from both the fami...,Stainless steel / Concrete / Wood,4 months,NaN,NaN,NaN,"Bodegas José Pariente started up in the 1960s,...",NaN,Stony / Sand,Continental with Atlantic influence,NaN,NaN,NaN,4.4,147 reviews,NaN,NaN,2021.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,2022,White,jose pariente,Rueda,NaN
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy note

In [114]:
# Remove extra characters in "customer_reviews_number" and convert to int type

vinissimus["customer_reviews_number"] = pd.to_numeric(vinissimus["customer_reviews_number"].str.replace(" reviews", "").str.replace(" review", ""), errors='coerce', downcast='integer')
vinissimus["customer_reviews_number"].value_counts()

0.0      2024
1.0      1081
2.0       585
3.0       334
4.0       246
         ... 
89.0        1
60.0        1
80.0        1
160.0       1
118.0       1
Name: customer_reviews_number, Length: 93, dtype: int64

### Type of agriculture

In [115]:
vinissimus["type_agriculture"].value_counts()

ORGANIC             676
PARKER              652
PEÑÍN               122
BIODYNAMIC           78
SUCKLING             69
JANCIS\nROBINSON     20
DECANTER             16
WINE\nSPECTATOR       3
Conventional          2
Name: type_agriculture, dtype: int64

In [116]:
# Keep only organic and biodynamic in the "type of agriculture", and assume that the rest are conventional agriculture

vinissimus["type_agriculture"] = vinissimus["type_agriculture"].apply(lambda x: "Organic" if x == "ORGANIC" else ("Biodynamic" if x == "BIODYNAMIC" else "Conventional"))
vinissimus["type_agriculture"].value_counts(dropna=False)

Conventional    4603
Organic          676
Biodynamic        78
Name: type_agriculture, dtype: int64

### Scores (Parker and Peñín)
The logic here is that we're are not interested in knowing if the wine obtained a score in previous years, we only want to keep the scores when the wine that received the score is the actual wine that we are seeing. <br>
Therefore we will replace the score year with NaN if it's not equal to the wine year. 

In [117]:
vinissimus["last_score_year"].value_counts(dropna=False)

NaN       3124
2020.0     393
2019.0     323
2018.0     231
2021.0     201
2016.0     181
2017.0     151
2015.0     130
2012.0     105
2014.0     100
2011.0      79
2010.0      70
2013.0      62
2009.0      61
2007.0      32
2008.0      31
2006.0      27
2005.0      19
2004.0      11
2022.0      10
2001.0       5
1926.0       2
1990.0       1
1980.0       1
1958.0       1
2003.0       1
2002.0       1
2000.0       1
1968.0       1
1996.0       1
1964.0       1
Name: last_score_year, dtype: int64

In [118]:
vinissimus.loc[vinissimus["year"] != vinissimus["last_score_year"] , "last_score_year"] = np.nan

In [119]:
vinissimus["last_score_year"].value_counts(dropna=False)

NaN       4718
2020.0     121
2019.0     108
2018.0      82
2021.0      68
2016.0      62
2017.0      49
2015.0      34
2014.0      28
2013.0      14
2012.0      13
2010.0      12
2011.0      12
2009.0       8
2022.0       4
2006.0       3
2004.0       3
2008.0       3
2001.0       3
2005.0       2
2007.0       2
1926.0       2
1980.0       1
1958.0       1
2002.0       1
1968.0       1
1996.0       1
1964.0       1
Name: last_score_year, dtype: int64

In [120]:
vinissimus_short = vinissimus[keep_cols]
vinissimus_short.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year,year,type_wine_details,winery_norm,region_gi
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,13.5,4.2,217.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,Conventional,93-94 PK,NaN,NaN,2021,Red,raul perez viticultor,Bierzo
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,11.5,4.4,45.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,Conventional,NaN,NaN,NaN,<NA>,Frizzante white,perelada,Wines without GI
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,11.5,4.8,95.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,Conventional,NaN,NaN,NaN,2021,White,masia bach,Catalunya
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,13.0,4.4,147.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,Conventional,NaN,NaN,NaN,2022,White,jose pariente,Rueda
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,13.5,4.4,94.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,Conventional,NaN,NaN,2018.0,2018,Red,el coto de rioja,Rioja


In [121]:
# Do some checks to make sure it's correct 

trying_score_year = vinissimus_short[vinissimus_short["last_score_year"].notna()] 
trying_score_year.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year,year,type_wine_details,winery_norm,region_gi
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,13.5,4.4,94.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,Conventional,NaN,NaN,2018.0,2018,Red,el coto de rioja,Rioja
8,El Coto Crianza 2019,9.8,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,El Coto Crianza is a rounded and fruit-forward...,Cherry red / Bright / Semi-opaque,Red fruit / Vanilla / Ageing notes,Silky / Spicy notes / Persistent / Silky tannins,16ºC,13.5,4.6,141.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/el-coto-cri...,A ruby-red colour and of medium density in app...,Sausages and hamStews and casserolesRed meatCu...,Conventional,NaN,NaN,2019.0,2019,Red,el coto de rioja,Rioja
10,Viña Ardanza Reserva 2016,27.8,0.75,Red,La Rioja,80% Tempranillo20% Garnacha,La Rioja Alta,Viña Ardanza is the Rioja Alta’s most well-kno...,Picota cherry red,Cherries / Cassis / Black pepper / Tobacco / N...,Fresh on entry / Flavoursome / Elegant tannins...,Serve between 17ºC and 18ºC,14.5,4.6,212.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/vina-ardanz...,"Viña Ardanza is a very fruit-forward, fresh an...",Red meatGameCured cheesesSausages and ham,Conventional,94+ PK,NaN,2016.0,2016,Red,la rioja alta,Rioja
11,Pago de Carraovejas 2020,36.4,0.75,Red,Castilla y León,93% Tempranillo4% Cabernet Sauvignon3% Merlot,Bodega Pago de Carraovejas,The Pago de Carraovejas is a genuine icon amon...,Purplish red / Purple rim,Ripe black fruit / Lactic notes / Balsamic aro...,Elegant / Structured / Profound / deep,Serve between 16ºC and 18ºC,15.0,4.8,120.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/pago-de-car...,A distinguished wine that appears a lovely pur...,Red meatCured cheesesPoultry and white meats,Conventional,93 PK,NaN,2020.0,2020,Red,pago de carraovejas,Ribera del Duero
16,Domaines Lupier La Dama 2018,40.6,0.75,Red,Navarra,100% Garnacha,Domaines Lupier,NaN,Picota cherry red / Intense / Opaque / Clean /...,Expressive / Higly aromatic / Fruit aromas / R...,Balanced / Powerful / Fresh / Fruit-forward / ...,Serve between 16ºC and 18ºC,14.0,0.0,0.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/domaines-lu...,NaN,NaN,Conventional,97 PK,NaN,2018.0,2018,Red,domaines lupier,Navarre


In [122]:
trying_score_year.describe()

,price,bottle,alcohol_content,customer_reviews,customer_reviews_number,last_score_year,year
count,639.000000,578.000000,637.000000,639.000000,639.000000,639.000000,605.0
mean,43.554930,0.737128,13.802512,2.728795,6.519562,2016.672926,2016.8
std,64.168705,0.082282,1.381320,2.170958,18.161825,7.397221,7.514533
min,9.400000,0.370000,9.500000,0.000000,0.000000,1926.000000,1926.0
25%,19.300000,0.750000,13.000000,0.000000,0.000000,2016.000000,2016.0
50%,27.200000,0.750000,14.000000,4.000000,1.000000,2018.000000,2018.0
75%,42.350000,0.750000,14.500000,4.600000,5.000000,2020.000000,2020.0
max,1203.400000,1.500000,22.000000,5.000000,212.000000,2022.000000,2022.0


Now, remove scores where we don't have a year, since they belong to previous years.

In [123]:
vinissimus["last_score_parker"].value_counts(dropna=False)

NaN          3954
91 PK         223
90 PK         222
92 PK         211
93 PK         202
94 PK         136
95 PK          95
93+ PK         48
96 PK          43
92+ PK         34
91+ PK         34
94+ PK         32
97 PK          31
90+ PK         18
98 PK          12
95+ PK         12
96+ PK          8
99 PK           5
100 PK          4
93-94 PK        3
91? PK          3
94-96 PK        3
97+ PK          2
90-92 PK        2
91-92 PK        2
98+ PK          2
99-100 PK       2
96-98 PK        2
95-96 PK        2
93? PK          1
99+ PK          1
97-100 PK       1
94-95 PK        1
93-95 PK        1
97-98 PK        1
89-91 PK        1
88-91 PK        1
91-93 PK        1
90-91 PK        1
Name: last_score_parker, dtype: int64

In [124]:
vinissimus.loc[pd.isna(vinissimus["last_score_year"]), "last_score_parker"] = np.nan
vinissimus["last_score_parker"].value_counts(dropna=False)

NaN         4910
93 PK         80
91 PK         68
92 PK         63
94 PK         56
95 PK         42
90 PK         37
96 PK         19
97 PK         12
94+ PK        11
92+ PK        10
93+ PK         9
91+ PK         8
98 PK          5
90+ PK         5
99 PK          4
96+ PK         3
95+ PK         3
94-96 PK       2
91? PK         2
97+ PK         2
100 PK         2
93-94 PK       1
97-98 PK       1
99+ PK         1
90-91 PK       1
Name: last_score_parker, dtype: int64

In [125]:
vinissimus_short = vinissimus[keep_cols]
vinissimus_short.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year,year,type_wine_details,winery_norm,region_gi
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,13.5,4.2,217.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,Conventional,NaN,NaN,NaN,2021,Red,raul perez viticultor,Bierzo
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,11.5,4.4,45.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,Conventional,NaN,NaN,NaN,<NA>,Frizzante white,perelada,Wines without GI
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,11.5,4.8,95.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,Conventional,NaN,NaN,NaN,2021,White,masia bach,Catalunya
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,13.0,4.4,147.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,Conventional,NaN,NaN,NaN,2022,White,jose pariente,Rueda
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,13.5,4.4,94.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,Conventional,NaN,NaN,2018.0,2018,Red,el coto de rioja,Rioja


In [126]:
vinissimus["last_score_penin"].value_counts(dropna=False)

NaN      4657
90 PN     229
91 PN     137
92 PN     115
93 PN     115
94 PN      56
95 PN      26
96 PN      15
97 PN       4
99 PN       2
98 PN       1
Name: last_score_penin, dtype: int64

In [127]:
vinissimus.loc[pd.isna(vinissimus["last_score_year"]), "last_score_penin"] = np.nan
vinissimus["last_score_penin"].value_counts(dropna=False)

NaN      5209
90 PN      34
93 PN      31
91 PN      27
92 PN      17
94 PN      17
95 PN       9
96 PN       8
97 PN       3
99 PN       2
Name: last_score_penin, dtype: int64

In [128]:
# Checks

vinissimus_short = vinissimus[keep_cols]
trying_scores = vinissimus_short[vinissimus_short["last_score_year"].isna()] 
trying_scores.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,last_score_parker,last_score_penin,last_score_year,year,type_wine_details,winery_norm,region_gi
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,13.5,4.2,217.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,Conventional,NaN,NaN,NaN,2021,Red,raul perez viticultor,Bierzo
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,11.5,4.4,45.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,Conventional,NaN,NaN,NaN,<NA>,Frizzante white,perelada,Wines without GI
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,11.5,4.8,95.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,Conventional,NaN,NaN,NaN,2021,White,masia bach,Catalunya
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,13.0,4.4,147.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,Conventional,NaN,NaN,NaN,2022,White,jose pariente,Rueda
5,René Barbier Semi Dulce 2021,8.8,0.75,White,Cataluña,40% Macabeo30% Parellada20% Xarel·lo10% Muscat...,René Barbier,NaN,Straw yellow / Pale,Fruit aromas / White fruit / Apple / Pear / Ba...,Smooth / Fresh / Light / Floral notes / Variet...,Serve between 7ºC and 9ºC,11.5,4.8,88.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/rene-barbie...,NaN,NaN,Conventional,NaN,NaN,NaN,2021,White,rene barbier,Catalunya


In [129]:
# All values should be NaN

print(trying_scores["last_score_parker"].value_counts(dropna=False))
print()
print(trying_scores["last_score_penin"].value_counts(dropna=False))

NaN    4718
Name: last_score_parker, dtype: int64

NaN    4718
Name: last_score_penin, dtype: int64


In [130]:
# Remove extra characters in the Parker score column

vinissimus["last_score_parker"].value_counts()

93 PK       80
91 PK       68
92 PK       63
94 PK       56
95 PK       42
90 PK       37
96 PK       19
97 PK       12
94+ PK      11
92+ PK      10
93+ PK       9
91+ PK       8
90+ PK       5
98 PK        5
99 PK        4
95+ PK       3
96+ PK       3
91? PK       2
94-96 PK     2
97+ PK       2
100 PK       2
93-94 PK     1
97-98 PK     1
99+ PK       1
90-91 PK     1
Name: last_score_parker, dtype: int64

Here we have to remove the "PK". We could replace the "+" with ".5" and the same for the ones that have a score between 2 points (i.e. 93-94 would be 93.5), but this would "alter" the score since there are no decimal values in the Parker score. Still, since it could be interesting for doing some analysis to have the column with numerical values, we will create a new column with the transformed data.

In [131]:
vinissimus["last_score_parker"] = vinissimus["last_score_parker"].str.replace(" PK", "").str.replace("?", "")
vinissimus["last_score_parker"].value_counts(dropna=False)

NaN      4910
93         80
91         70
92         63
94         56
95         42
90         37
96         19
97         12
94+        11
92+        10
93+         9
91+         8
90+         5
98          5
99          4
96+         3
95+         3
94-96       2
97+         2
100         2
93-94       1
97-98       1
99+         1
90-91       1
Name: last_score_parker, dtype: int64

In [132]:
vinissimus["parker_score_num"] = vinissimus["last_score_parker"].str.replace("+", ".5", regex=True)
vinissimus["parker_score_num"] = vinissimus["parker_score_num"].apply(lambda x: x if pd.isna(x) else (f"{x[:2]}.5" if len(x)==5 else x))
vinissimus["parker_score_num"] = vinissimus["parker_score_num"].apply(lambda x: float(x) if pd.notna(x) else x)
vinissimus["parker_score_num"].value_counts()

93.0     80
91.0     70
92.0     63
94.0     56
95.0     42
90.0     37
96.0     19
94.5     13
97.0     12
92.5     10
93.5     10
91.5      8
90.5      6
98.0      5
99.0      4
95.5      3
96.5      3
97.5      3
100.0     2
99.5      1
Name: parker_score_num, dtype: int64

In [133]:
# Remove extra characters in the Peñín score column

vinissimus["last_score_penin"].value_counts()

90 PN    34
93 PN    31
91 PN    27
92 PN    17
94 PN    17
95 PN     9
96 PN     8
97 PN     3
99 PN     2
Name: last_score_penin, dtype: int64

In [134]:
vinissimus["last_score_penin"] = vinissimus["last_score_penin"].str.replace(" PN", "")
vinissimus["last_score_penin"] = vinissimus["last_score_penin"].apply(lambda x: int(x) if pd.notna(x) else x)
vinissimus["last_score_penin"].value_counts()

90.0    34
93.0    31
91.0    27
92.0    17
94.0    17
95.0     9
96.0     8
97.0     3
99.0     2
Name: last_score_penin, dtype: int64

In [135]:
vinissimus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5357 entries, 0 to 5356
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   wine                        5357 non-null   object 
 1   price                       5357 non-null   float64
 2   bottle                      4794 non-null   float64
 3   type_wine                   5357 non-null   object 
 4   region                      5357 non-null   object 
 5   grapes                      5337 non-null   object 
 6   winery                      5357 non-null   object 
 7   wine_description            2805 non-null   object 
 8   tasting_descr               1175 non-null   object 
 9   tasting_view                3823 non-null   object 
 10  tasting_bouquet             3823 non-null   object 
 11  tasting_mouth               3823 non-null   object 
 12  light_bold_perc             381 non-null    float64
 13  smooth_tannic_perc          381 n

Now that the scores columns are ok, the last_score_year column can be dropped, and change the name of the scores columns.

In [136]:
vinissimus = vinissimus.rename(columns={"last_score_parker":"parker_score", "last_score_penin":"penin_score"})

In [137]:
vinissimus.columns

Index(['wine', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'winery',
       'wine_description', 'tasting_descr', 'tasting_view', 'tasting_bouquet',
       'tasting_mouth', 'light_bold_perc', 'smooth_tannic_perc',
       'dry_sweet_perc', 'soft_acidic_perc', 'serving_storage_conditions',
       'serve_temperature', 'food_pairing', 'allergens', 'alcohol_content',
       'production', 'winemaking_info', 'vessel_material', 'aging_period',
       'casks_age', 'casks_wood', 'more_winemaking_info', 'vineyards',
       'type_agriculture', 'vineyard_age', 'soil_type', 'climate',
       'orientation', 'surface', 'customer_reviews', 'customer_reviews_number',
       'parker_score', 'penin_score', 'last_score_year', 'image', 'url',
       'year', 'type_wine_details', 'winery_norm', 'region_gi',
       'tasting_notes', 'parker_score_num'],
      dtype='object')

In [138]:
remove_cols = ["last_score_year", "last_score_parker", "last_score_penin"]
add_cols = ["parker_score", "parker_score_num", "penin_score"]

keep_cols = keep_cols + add_cols
keep_cols = [x for x in keep_cols if x not in remove_cols]

keep_cols

['wine',
 'price',
 'bottle',
 'type_wine',
 'region',
 'grapes',
 'winery',
 'wine_description',
 'tasting_view',
 'tasting_bouquet',
 'tasting_mouth',
 'serve_temperature',
 'alcohol_content',
 'customer_reviews',
 'customer_reviews_number',
 'image',
 'url',
 'tasting_descr',
 'food_pairing',
 'type_agriculture',
 'year',
 'type_wine_details',
 'winery_norm',
 'region_gi',
 'parker_score',
 'parker_score_num',
 'penin_score']

### Grapes

In [139]:
vinissimus["grapes"].value_counts()

100% Tempranillo                                          649
100% Garnacha                                             319
100% Verdejo                                              166
100% Albariño                                             164
100% Xarel·lo                                             117
                                                         ... 
30% Garnacha30% Syrah30% Cabernet Sauvignon10% Macabeo      1
80% Garnacha2% White Grenache                               1
85% Riesling15% Albariño                                    1
50% Macabeo50% Sauvignon Blanc                              1
MencíaTempranillo                                           1
Name: grapes, Length: 1710, dtype: int64

In [140]:
# Remove numbers and percentage symbols from the grapes column

vinissimus["grapes"] = vinissimus["grapes"].str.replace(r'\d+% ', ', ', regex=True)
vinissimus["grapes"].value_counts()

, Tempranillo                                                   649
, Garnacha                                                      319
, Verdejo                                                       166
, Albariño                                                      164
, Xarel·lo                                                      117
                                                               ... 
TempranilloMerlotCabernet SauvignonMoristelParraleta              1
GarnachaGarnacha PeludaSyrahCariñenaCabernet SauvignonMerlot      1
BrancellaoFerrolCaiño Tinto                                       1
, Merseguera, Malvasia                                            1
MencíaTempranillo                                                 1
Name: grapes, Length: 1332, dtype: int64

In [141]:
# Remove commas when they are at the beginning of the value

def remove_commas(word):
    if word[:2] == ", ":
        return word[2:]
    else:
        return word

vinissimus["grapes"] = vinissimus["grapes"].apply(lambda x: remove_commas(x) if pd.notna(x) else x)
vinissimus["grapes"].value_counts()

Tempranillo                                                     649
Garnacha                                                        319
Verdejo                                                         166
Albariño                                                        165
Xarel·lo                                                        117
                                                               ... 
GarnachaGarnacha PeludaSyrahCariñenaCabernet SauvignonMerlot      1
BrancellaoFerrolCaiño Tinto                                       1
Merseguera, Malvasia                                              1
SousónMenciaPedral                                                1
MencíaTempranillo                                                 1
Name: grapes, Length: 1317, dtype: int64

In [142]:
# Add spaces and commas when the grape names are all together 

vinissimus["grapes"] = vinissimus["grapes"].str.replace(r"(\w)([A-Z])", r"\1, \2", regex=True)
vinissimus["grapes"].value_counts()

Tempranillo                                         649
Garnacha                                            319
Verdejo                                             166
Albariño                                            165
Xarel·lo                                            117
                                                   ... 
Sumoll, Garnacha, Marcelan, Syrah                     1
Morenillo, Cabernet Sauvignon                         1
Tempranillo, Cabernet Sauvignon, Garnacha, Syrah      1
Alicante Bouschet, Monastrell                         1
Mencía, Tempranillo                                   1
Name: grapes, Length: 1152, dtype: int64

In [143]:
vinissimus["grapes"].nunique()

1152

In order to have a count of the number of wines that have each grape (ie: Tempranillo, 500 wines), we will use pandas explode. This function transforms each element of a list-like to a row, replicating index values.

In [144]:
vinissimus["grapes"] = vinissimus["grapes"].str.split(", ") # turns each value into a list
vinissimus_exploded = vinissimus.explode("grapes")
vinissimus_exploded.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,parker_score,penin_score,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes,parker_score_num
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,Conventional,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2,217.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021,Red,raul perez viticultor,Bierzo,NaN,NaN
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,Macabeo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Wines without GI,NaN,NaN
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,Parellada,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Wines without GI,NaN,NaN
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Wines without GI,NaN,NaN
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,Xarel·lo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5,NaN,NaN,6 months,American oak,

In [145]:
vinissimus_exploded["grapes"].value_counts()

Tempranillo           1274
Garnacha              1085
Cabernet Sauvignon     508
Syrah                  496
Xarel·lo               455
                      ... 
Cribatinaja              1
Provechón                1
Robal                    1
Grenache                 1
Mencía                   1
Name: grapes, Length: 198, dtype: int64

#### Create new feature: varietal/blend
Varietal wines are made of a single grape variety (although it is accepted to have 15% of other grapes without naming them in the label), while blends are mixes of more than one grape.

In [146]:
vinissimus["grapes"] = vinissimus["grapes"].fillna("Unknown")

In [147]:
vinissimus["num_varieties"] = "Varietal"
mask = vinissimus["grapes"].apply(len) > 1
vinissimus.loc[mask, "num_varieties"] = "Blend"
vinissimus["num_varieties"].value_counts(dropna=False)

Varietal    3060
Blend       2297
Name: num_varieties, dtype: int64

In [148]:
keep_cols.append("num_varieties")

# Give format to the final dataframe

In [149]:
vinissimus.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,allergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,parker_score,penin_score,last_score_year,image,url,year,type_wine_details,winery_norm,region_gi,tasting_notes,parker_score_num,num_varieties
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,[Mencia],Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,60.0,50.0,20.0,60.0,This is a lovely wine which can be drunk as so...,Serve between 14ºC and 16ºC,Sausages and hamRed meatStews and casseroles,Contains sulfites,13.5,"110,000 bottles",Raúl Pérez produces all his wines with whole b...,Wood / Cement,Between 10 and 12 months,Used,French oak,Bottled unfined and unfiltered,Villegas is one of the vineyards most highly r...,Conventional,Between 30 and 130 years,Sand,Continental with Atlantic influence,East and south,2.00 hectares,4.2,217.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,2021,Red,raul perez viticultor,Bierzo,NaN,NaN,Varietal
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,"[Macabeo, Parellada, Xarel·lo]",Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,NaN,NaN,NaN,The prestigious and successful Perelada winery...,NaN,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,<NA>,Frizzante white,perelada,Wines without GI,NaN,NaN,Blend
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,"[Xarel·lo, Macabeo]",Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,NaN,NaN,NaN,NaN,NaN,Snacks and appetizersStarters and suppersSeafo...,Contains sulfites,11.5,NaN,NaN,6 months,American oak,NaN,NaN,NaN,NaN,Conventional,NaN,NaN,NaN,NaN,NaN,4.8,95.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,2021,White,masia bach,Catalunya,NaN,NaN,Blend
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,[Verdejo],Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,40.0,30.0,70.0,NaN,"Due to its freshness and acidity, it works ver...",Serve between 6ºC and 8ºC,FishSaladsStarters and suppersVegetablesSoft c...,Contains sulfites,13.0,NaN,It is produced using grapes from both the fami...,Stainless steel / Concrete / Wood,4 months,NaN,NaN,NaN,"Bodegas José Pariente started up in the 1960s,...",Conventional,Stony / Sand,Continental with Atlantic influence,NaN,NaN,NaN,4.4,147.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,2022,White,jose pariente,Rueda,NaN,NaN,Varietal
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,[Tempranillo],El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...

In [150]:
vinissimus_short = vinissimus[keep_cols]
vinissimus_short.head()

,wine,price,bottle,type_wine,region,grapes,winery,wine_description,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,alcohol_content,customer_reviews,customer_reviews_number,image,url,tasting_descr,food_pairing,type_agriculture,year,type_wine_details,winery_norm,region_gi,parker_score,parker_score_num,penin_score,num_varieties
0,Ultreia Saint Jacques 2021,16.2,0.75,Red,Castilla y León,[Mencia],Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,Intense,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,Serve between 14ºC and 16ºC,13.5,4.2,217.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,"Bottled unclarified and unfiltered, Ultreia Sa...",Sausages and hamRed meatStews and casseroles,Conventional,2021,Red,raul perez viticultor,Bierzo,NaN,NaN,NaN,Varietal
1,Blanc Pescador,7.8,0.75,Sparkling,Wines without GI,"[Macabeo, Parellada, Xarel·lo]",Perelada,Blanc Pescador and Cresta Rosa have been bench...,Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,NaN,11.5,4.4,45.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,"A pale colour, with light yellow-green hues an...",Snacks and appetizersStarters and suppersSalad...,Conventional,<NA>,Frizzante white,perelada,Wines without GI,NaN,NaN,NaN,Blend
2,Bach Extrísimo Blanco Semidulce 2021,9.4,0.75,White,Cataluña,"[Xarel·lo, Macabeo]",Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,Yellow / Golden glints / Clean / Bright,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...,NaN,11.5,4.8,95.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,"In 1915, brothers Pere and Ramon Bach i Escope...",Snacks and appetizersStarters and suppersSeafo...,Conventional,2021,White,masia bach,Catalunya,NaN,NaN,NaN,Blend
3,José Pariente Verdejo 2022,16.2,0.75,White,Castilla y León,[Verdejo],Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,Straw yellow / Greenish glints,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,Serve between 6ºC and 8ºC,13.0,4.4,147.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...,A golden colour in the glass with greenish hig...,FishSaladsStarters and suppersVegetablesSoft c...,Conventional,2022,White,jose pariente,Rueda,NaN,NaN,NaN,Varietal
4,Coto de Imaz Reserva 2018,13.0,0.75,Red,La Rioja,[Tempranillo],El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,Picota cherry / Intense / Garnet rim,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,Serve between 16ºC and 18ºC,13.5,4.4,94.0,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...,"At first sight, it appears a vivid cherry colo...",Sausages and hamRed meatGameCured cheeses,Conventional,2018,Red,el coto de rioja,Rioja,NaN,NaN,NaN,Varietal


In [151]:
vinissimus_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5357 entries, 0 to 5356
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   wine                     5357 non-null   object 
 1   price                    5357 non-null   float64
 2   bottle                   4794 non-null   float64
 3   type_wine                5357 non-null   object 
 4   region                   5357 non-null   object 
 5   grapes                   5357 non-null   object 
 6   winery                   5357 non-null   object 
 7   wine_description         2805 non-null   object 
 8   tasting_view             3823 non-null   object 
 9   tasting_bouquet          3823 non-null   object 
 10  tasting_mouth            3823 non-null   object 
 11  serve_temperature        3712 non-null   object 
 12  alcohol_content          5334 non-null   float64
 13  customer_reviews         5355 non-null   float64
 14  customer_reviews_number 

In [152]:
vinissimus_short.columns

Index(['wine', 'price', 'bottle', 'type_wine', 'region', 'grapes', 'winery',
       'wine_description', 'tasting_view', 'tasting_bouquet', 'tasting_mouth',
       'serve_temperature', 'alcohol_content', 'customer_reviews',
       'customer_reviews_number', 'image', 'url', 'tasting_descr',
       'food_pairing', 'type_agriculture', 'year', 'type_wine_details',
       'winery_norm', 'region_gi', 'parker_score', 'parker_score_num',
       'penin_score', 'num_varieties'],
      dtype='object')

In [153]:
len(vinissimus_short.columns)

28

In [154]:
# Reorder columns

new_order = ['wine', 'year', 'winery', 'winery_norm', 'price', 'bottle', 'type_wine', 'type_wine_details',
             'type_agriculture', 'region', 'region_gi', 'grapes', 'num_varieties', 'wine_description', 'tasting_descr',
             'tasting_view', 'tasting_bouquet', 'tasting_mouth', 'serve_temperature',
             'food_pairing', 'alcohol_content', 'customer_reviews', 'customer_reviews_number',
             'parker_score', 'parker_score_num', 'penin_score', 'image', 'url']

vinissimus_short = vinissimus[new_order]

In [155]:
len(new_order)

28

In [156]:
vinissimus_short.sample(10)

,wine,year,winery,winery_norm,price,bottle,type_wine,type_wine_details,type_agriculture,region,region_gi,grapes,num_varieties,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,food_pairing,alcohol_content,customer_reviews,customer_reviews_number,parker_score,parker_score_num,penin_score,image,url
442,Vermell 2021,2021,Celler del Roure,celler del roure,11.0,0.75,Red,Red,Conventional,Comunidad Valenciana,Valencia,"[Monastrell, Garnacha, Mandó]",Blend,Vermell is a very special little wine. It's ma...,NaN,NaN,NaN,NaN,Serve between 15ºC and 17ºC,NaN,13.0,4.2,40.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/parotet-ver...
2302,Ultreia La Claudina 2021,2021,Raúl Pérez Viticultor,raul perez viticultor,52.4,0.75,White,White,Conventional,Castilla y León,Bierzo,[Godello],Varietal,NaN,NaN,Deep yellow / Intense,Floral scents / Dried fruit and nuts / Cereal ...,Rounded / Good acidity / Complex / Elegant / W...,Serve between 8ºC and 10ºC,NaN,13.5,5.0,3.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-la-...
2723,Petit Caligo 2014 (0.37 L),<NA>,DG Viticultors,dg viticultors,23.2,0.37,Dessert,Noble Rot,Conventional,Cataluña,Penedès,"[Sauvignon Blanc, Chardonnay]",Blend,NaN,NaN,Pale gold,Lemon peel / Peach / Spices / Floral notes / D...,Sweetness and acidity balanced / Long,Serve between 8ºC and 10ºC,NaN,14.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/petit-caligo/
2507,Celeste Roble 2022,2022,Pago del Cielo-Torres,pago del cielotorres,13.0,0.75,Red,Red,Conventional,Castilla y León,Ribera del Duero,[Tempranillo],Varietal,In 2003 the Familia Torres wine-producers made...,NaN,Cherry red / Luminous,Fruit aromas / Blackberry jam / Figs / Plums,Velvety / Flavoursome / Subtle,Serve between 14ºC and 16ºC,NaN,14.0,4.3,16.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/celeste-roble/
4247,Evodia Pizarras Antiguas 2017,2017,Bodegas San Alejandro,san alejandro,16.5,0.75,Red,Red,Conventional,Aragón,Calatayud,[Garnacha],Varietal,NaN,NaN,mulberry red / Dark / Violet glints,Higly aromatic / Fruit aromas / Ripe black fru...,Good acidity / Fine tannins / Fruit aromas / W...,Serve between 16ºC and 18ºC,NaN,15.0,4.0,1.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/evodia-piza...
445,Pedrabona 2021,2021,Josep Grau Viticultor,josep grau viticultor,28.0,0.75,Red,Red,Organic,Cataluña,Priorat,"[Cariñena, Garnacha]",Blend,The grape comes from a farm of 4 hectares of B...,Josep Grau Viticultor is the personal project ...,NaN,NaN,NaN,16ºC,NaN,15.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/pedrabona/
106,Cava Bach Rosé Brut,<NA>,Masia Bach,masia bach,9.4,0.75,Sparkling,Cava,Conventional,Cataluña,DO Cava,"[Garnacha, Monastrell, Pinot Noir]",Blend,"In 1915, brothers Pere and Ramon Bach i Escope...",NaN,Pale pink / Quite intense / Fine bubbles,Pleasant / Fruit aromas / Ripe red fruit / Str...,Balanced / Smooth / Pleasant,Serve between 6ºC and 8ºC,NaN,12.0,4.5,35.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/cava-bach-r...
1534,Nabuc Blanco Semi-dulce,<NA>,Vins Padró,vins padro,10.0,0.75,White,White,Conventional,Wines without GI,Wines without GI,"[White Grenache, Muscat of Alexandria]",Blend,NaN,NaN,Yellow,Fruity / Pear / Melon / Camomile tea,Sweet / Fresh / Unctuous / Persistent,Serve between 4ºC and 6ºC,NaN,12.0,4.7,3.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/nabuc-blanc...
1848,Pinord NOX Rosado Seducción 2022,2022,Bodegas Pinord,pinord,10.2,NaN,Rosé,Rosé,Conventional,Cataluña,Penedès,"[Cabernet Sauvignon, Merlot, Tempranillo]",Blend,NaN,NaN,Cherry pink / Intense / Reddish glints / Bright,Hi

In [157]:
vinissimus_short.shape

(5357, 28)

#### Check for duplicates again, now that there are less features
Duplicated() returns an error because the "grapes" column is a list (an unhashable type). To solve it, we will create a copy of the dataframe and convert the lists to tuples, which are hashable.

In [158]:
check_duplicates = vinissimus_short.copy()
check_duplicates["grapes"] = check_duplicates["grapes"].fillna('').apply(lambda x: tuple(x) if isinstance(x, list) else x)

check_duplicates.duplicated().value_counts()

False    5357
dtype: int64

In [159]:
duplicated_indices = check_duplicates[check_duplicates.duplicated(keep='first')].index 
vinissimus_short.drop(duplicated_indices, inplace=True)

In [160]:
vinissimus_short.shape

(5357, 28)

In [161]:
vinissimus_short["wine"].nunique()

5344

Some of the wine names are duplicated. 

In [162]:
name_dupl = vinissimus_short[vinissimus_short.duplicated(subset="wine", keep=False)]
name_dupl

,wine,year,winery,winery_norm,price,bottle,type_wine,type_wine_details,type_agriculture,region,region_gi,grapes,num_varieties,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,food_pairing,alcohol_content,customer_reviews,customer_reviews_number,parker_score,parker_score_num,penin_score,image,url
1068,El Jardín de las Iguales Macabeo 2016,2016,Bodegas Frontonio,frontonio,160.8,NaN,White,White,Conventional,Wines without GI,Wines without GI,[Macabeo],Varietal,Bodegas Frontonio is named after Saint Fronton...,NaN,Yellow / Steely glints,Citrus fruits / Lemon / Clove / Thyme / Fruit ...,Refreshing acidity / Elegant / Vivacious / Spi...,NaN,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/el-jardin-d...
1119,Marco Valerio Marcial 2018,2018,Bodegas Langa,langa,22.8,NaN,Red,Red,Conventional,Aragón,Calatayud,[Garnacha],Varietal,NaN,NaN,Ripe Picota cherry red / Purplish glints,Higly aromatic / Floral scents / Red flowers /...,Good entry / Smooth tannins / Elegant / Balsam...,Serve between 17ºC and 18ºC,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/marco-v-m/
1195,Marco Valerio Marcial 2018,2018,Bodegas Langa,langa,22.8,0.75,Red,Red,Conventional,Aragón,Calatayud,[Garnacha],Varietal,NaN,NaN,Ripe Picota cherry red / Purplish glints,Higly aromatic / Floral scents / Red flowers /...,Good entry / Smooth tannins / Elegant / Balsam...,Serve between 17ºC and 18ºC,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/marco-v-m/
1258,Finca Els Camps Ull de Llebre 2014,2014,Jané Ventura,jane ventura,24.2,0.75,Red,Red,Conventional,Cataluña,Penedès,[Tempranillo],Varietal,NaN,NaN,NaN,NaN,NaN,Serve between 10ºC and 12ºC,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/finca-els-c...
1316,Riu de Gost Incrocio Manzoni 2019,2019,Pla de Morei,pla de morei,34.0,0.75,White,White,Organic,Wines without GI,Wines without GI,[Incrocio Manzoni],Varietal,NaN,NaN,Old gold / Intense,Higly aromatic / Expressive / Exotic fruit / C...,Elegant / Dense / Creamy / Mineral / Long,NaN,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/riu-de-gost...
1448,Riu de Gost Incrocio Manzoni 2019,2019,Pla de Morei,pla de morei,34.0,NaN,White,White,Organic,Wines without GI,Wines without GI,[Incrocio Manzoni],Varietal,NaN,NaN,Old gold / Intense,Higly aromatic / Expressive / Exotic fruit / C...,Elegant / Dense / Creamy / Mineral / Long,NaN,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/riu-de-gost...
1464,El Jardín de las Iguales Macabeo 2016,2016,Bodegas Frontonio,frontonio,160.8,0.75,White,White,Conventional,Wines without GI,Wines without GI,[Macabeo],Varietal,Bodegas Frontonio is named after Saint Fronton...,NaN,Yellow / Steely glints,Citrus fruits / Lemon / Clove / Thyme / Fruit ...,Refreshing acidity / Elegant / Vivacious / Spi...,NaN,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/el-jardin-d...
1465,Finca Els Camps Ull de Llebre 2014,2014,Jané Ventura,jane ventura,24.2,NaN,Red,Red,Conventional,Cataluña,Penedès,[Tempranillo],Varietal,NaN,NaN,NaN,NaN,NaN,Serve between 10ºC and 12ºC,NaN,13.5,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/finca-els-c...
1469,Fulanito 2020,2020,Wine Side Story,wine side story,11.2,NaN,Red,Red,Conventional,Castilla y León,Ribera del Duero,"[Tempranillo, Cabernet Sauvignon, Merlot]",Blend,The perfect wine for having an enjoyable drink...,Cuvée 3000 is a well-known Barcelona company d...,NaN,NaN,NaN,Serve between 15ºC and 17ºC,NaN,14.0,3.0,3.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https:

Most of the duplicated wines seem to be because there is a row with NaN in "bottle" and a row with the actual value. We will drop those rows with NaN values.

In [163]:
# Obtain the indexes of the duplicates with null values in "bottle"
nan_duplicates_index = name_dupl[name_dupl["bottle"].isna()].index
nan_duplicates_index

Int64Index([1068, 1119, 1448, 1465, 1469, 1640, 2147, 2156, 2160, 2309, 2313], dtype='int64')

In [164]:
# Drop those rows
vinissimus_short = vinissimus_short.drop(nan_duplicates_index)

In [165]:
len(vinissimus_short)

5346

In [166]:
vinissimus_short[vinissimus_short.duplicated(subset="wine", keep=False)]

,wine,year,winery,winery_norm,price,bottle,type_wine,type_wine_details,type_agriculture,region,region_gi,grapes,num_varieties,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,serve_temperature,food_pairing,alcohol_content,customer_reviews,customer_reviews_number,parker_score,parker_score_num,penin_score,image,url
1561,Mastinell Gisele 2022,2022,Heretat Mastinell,heretat mastinell,18.6,0.75,White,White,Organic,Cataluña,Penedès,[Chardonnay],Varietal,NaN,NaN,Yellow / Intense / Amber glints,Complex / Fresh / Citrus fruits / Grapefruit /...,Fruit aromas / Mango / Peach / Lactic notes,Serve between 7ºC and 9ºC,NaN,13.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/mas-tinell-...
2430,Lustau Manzanilla Pasada Manuel Cuevas (0.5 L),<NA>,Emilio Lustau,emilio lustau,24.8,0.50,Dessert,Manzanilla sherry,Conventional,Andalucía,Jerez-Manzanilla,[Palomino Fino],Varietal,The original company was founded in 1896 by Do...,NaN,NaN,NaN,NaN,Serve between 7ºC and 9ºC,NaN,17.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/lustau-manz...
3006,Mastinell Gisele 2022,2022,Heretat Mastinell,heretat mastinell,23.8,0.75,White,White,Conventional,Cataluña,Penedès,[Xarel·lo],Varietal,NaN,NaN,Pale yellow,Ageing notes / Toasty notes / Roasted notes / ...,Full / Flavoursome / Fruit aromas / Apple / Me...,Serve between 8ºC and 10ºC,NaN,13.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/mas-tinell-...
5054,Lustau Manzanilla Pasada Manuel Cuevas (0.5 L),<NA>,Emilio Lustau,emilio lustau,26.2,0.50,Dessert,Manzanilla sherry,Conventional,Andalucía,Jerez-Manzanilla,[Palomino Fino],Varietal,The original company was founded in 1896 by Do...,NaN,NaN,NaN,NaN,NaN,NaN,17.0,0.0,0.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/lustau-manz...


There are two wines left, the first one is not a duplicate, the wines are different even when they have the same name (maybe it's an error in the name?). The second one, the price is different. After checking the website, we will remove the 5054.

In [168]:
vinissimus_short = vinissimus_short.drop(5054)
len(vinissimus_short)

5345

The length of the dataset now is consistent with the unique names of wines. <br>
Last step, reset the index.

In [169]:
vinissimus_short = vinissimus_short.reset_index(drop=True)

# Cleaning the text data
#### Feature selection

In [170]:
nulls = pd.DataFrame(vinissimus_short.isna().sum()*100/len(vinissimus_short), columns=['percentage'])
nulls[nulls["percentage"]>0].sort_values('percentage', ascending = False)

,percentage
penin_score,97.231057
food_pairing,96.108513
parker_score,91.637044
parker_score_num,91.637044
tasting_descr,78.072965
wine_description,47.689429
serve_temperature,30.682881
tasting_view,28.606174
tasting_bouquet,28.606174
tasting_mouth,28.606174


Regarding the tasting features, we have a 78% of NaNs in the "tasting description" column, and a 28% of NaNs in the view, bouquet and mouth columns. <br>
725 wines have the 4 tasting columns. The rest have NaN in at least one.
- 3816 wines have tasting view, bouquet, mouth values. The text of these features is quite schematic, which is positive for our model.
- 450 wines have NaN in these columns, but have the tasting description. The text in this case is much more complex, it is a regular paragraph that can have extra information about the winery, etc, not related only with the wine itself.<br>

We will not try to add the info from the tasting description column since it would only add a roughly 10% extra of wines and it considerably increases the complexity of the text data. <br>
From the other 3, we will use the bouquet and the mouth, but not the view. Of the three parts of the tasting, amateurs tend to give less importance to how the wine looks or its particular colour (unless there is a defect in the wine, but that will not appear in the store's tasting notes), and they give more importance to how it tastes and how it smells. 

In [171]:
drop_cols = ['winery_norm', 'wine_description', 'tasting_descr',
       'tasting_view', 'serve_temperature',
       'food_pairing']
vinissimus_short = vinissimus_short.drop(drop_cols, axis=1)

#### Normalize
Since our data has a special format (eg: "Aromatic / Fruit aromas / Highly aromatic"), we can easily join the words in the terms that are consisted of more than one word. We will also remove stopwords (these are commonly used words that don't have an "important meaning", such as "the", "is"....), and convert to lower case.

In [172]:
stop_words = set(stopwords.words('english'))

def normalize_tasting_notes(text):
    if pd.isna(text):
        normalized = np.nan
    else:
        # Split by '/' and strip leading/trailing spaces from each part
        parts = [part.strip() for part in text.split('/')]
        # Replace spaces with underscores in each part, and remove stop words
        parts = [part.replace(" ", "_") for part in parts if part.lower() not in stop_words]
        # Join the parts with spaces to form the final string and convert to lowercase
        normalized = " ".join(parts).lower()

    return normalized

We will start with the columns that have more potential for building the recommender: tasting bouquet and tasting mouth. 

In [173]:
vinissimus_short[['wine', 'tasting_bouquet', 'tasting_mouth']].head()

,wine,tasting_bouquet,tasting_mouth
0,Ultreia Saint Jacques 2021,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied
1,Blanc Pescador,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling
2,Bach Extrísimo Blanco Semidulce 2021,Higly aromatic / Complex / Fruit aromas / Flor...,Good entry / Structured / Balanced / Good acid...
3,José Pariente Verdejo 2022,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness
4,Coto de Imaz Reserva 2018,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety


In [174]:
tasting_cols = ['tasting_bouquet', 'tasting_mouth']

text_data = vinissimus_short.dropna(subset=tasting_cols, how='all')
text_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3816 entries, 0 to 5342
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   wine                     3816 non-null   object 
 1   year                     3438 non-null   Int64  
 2   winery                   3816 non-null   object 
 3   price                    3816 non-null   float64
 4   bottle                   3435 non-null   float64
 5   type_wine                3816 non-null   object 
 6   type_wine_details        3816 non-null   object 
 7   type_agriculture         3816 non-null   object 
 8   region                   3816 non-null   object 
 9   region_gi                3816 non-null   object 
 10  grapes                   3816 non-null   object 
 11  num_varieties            3816 non-null   object 
 12  tasting_bouquet          3816 non-null   object 
 13  tasting_mouth            3816 non-null   object 
 14  alcohol_content         

### Bouquet

In [175]:
# Replace typo in "Higly aromatic" because it's driving me crazy
vinissimus_short["tasting_bouquet"] = vinissimus_short["tasting_bouquet"].str.replace("Higly", "Highly")
vinissimus_short[tasting_cols].head()

,tasting_bouquet,tasting_mouth
0,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied
1,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling
2,Highly aromatic / Complex / Fruit aromas / Flo...,Good entry / Structured / Balanced / Good acid...
3,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness
4,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety


In [176]:
vinissimus_short["bouquet_norm"] = vinissimus_short["tasting_bouquet"].apply(normalize_tasting_notes)
tasting_cols.append("bouquet_norm")
vinissimus_short[tasting_cols].head()

,tasting_bouquet,tasting_mouth,bouquet_norm
0,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,raspberry violets
1,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling,aromatic fruit_aromas highly_aromatic
2,Highly aromatic / Complex / Fruit aromas / Flo...,Good entry / Structured / Balanced / Good acid...,highly_aromatic complex fruit_aromas floral_sc...
3,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,white_fruit apricot fennel aniseed
4,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,ripe_fruit spicy_notes complex


### Mouth

In [177]:
vinissimus_short["mouth_norm"] = vinissimus_short["tasting_mouth"].apply(normalize_tasting_notes)
tasting_cols.append("mouth_norm")
vinissimus_short[tasting_cols].head()

,tasting_bouquet,tasting_mouth,bouquet_norm,mouth_norm
0,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,raspberry violets,red_fruit mineral voluminous medium-bodied
1,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling,aromatic fruit_aromas highly_aromatic,light pleasant sparkling
2,Highly aromatic / Complex / Fruit aromas / Flo...,Good entry / Structured / Balanced / Good acid...,highly_aromatic complex fruit_aromas floral_sc...,good_entry structured balanced good_acidity fl...
3,White fruit / Apricot / Fennel / Aniseed,Rich / Unctuous / Pleasant bitterness,white_fruit apricot fennel aniseed,rich unctuous pleasant_bitterness
4,Ripe fruit / Spicy notes / Complex,Fleshy / Powerful / Ripe tannins / Velvety,ripe_fruit spicy_notes complex,fleshy powerful ripe_tannins velvety


### Duplicates

In [178]:
check_duplicates = vinissimus_short.copy()
check_duplicates["grapes"] = check_duplicates["grapes"].fillna('').apply(lambda x: tuple(x) if isinstance(x, list) else x)

check_duplicates.duplicated().value_counts()

False    5345
dtype: int64

### Reset index

In [179]:
vinissimus_short = vinissimus_short.reset_index(drop=True)

### Save csv

In [180]:
vinissimus_short.to_csv("vinissimus_short_cleaned.csv", index=False)

### Explode the dataframe 
We will use it to analyse the grapes feature in the EDA step.

In [181]:
vinissimus_short_exploded = vinissimus_short.explode("grapes")
vinissimus_short_exploded.head()

,wine,year,winery,price,bottle,type_wine,type_wine_details,type_agriculture,region,region_gi,grapes,num_varieties,tasting_bouquet,tasting_mouth,alcohol_content,customer_reviews,customer_reviews_number,parker_score,parker_score_num,penin_score,image,url,bouquet_norm,mouth_norm
0,Ultreia Saint Jacques 2021,2021,Raúl Pérez Viticultor,16.2,0.75,Red,Red,Conventional,Castilla y León,Bierzo,Mencia,Varietal,Raspberry / Violets,Red fruit / Mineral / Voluminous / Medium-bodied,13.5,4.2,217.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...,raspberry violets,red_fruit mineral voluminous medium-bodied
1,Blanc Pescador,<NA>,Perelada,7.8,0.75,Sparkling,Frizzante white,Conventional,Wines without GI,Wines without GI,Macabeo,Blend,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling,11.5,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,aromatic fruit_aromas highly_aromatic,light pleasant sparkling
1,Blanc Pescador,<NA>,Perelada,7.8,0.75,Sparkling,Frizzante white,Conventional,Wines without GI,Wines without GI,Parellada,Blend,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling,11.5,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,aromatic fruit_aromas highly_aromatic,light pleasant sparkling
1,Blanc Pescador,<NA>,Perelada,7.8,0.75,Sparkling,Frizzante white,Conventional,Wines without GI,Wines without GI,Xarel·lo,Blend,Aromatic / Fruit aromas / Highly aromatic,Light / Pleasant / Sparkling,11.5,4.4,45.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...,aromatic fruit_aromas highly_aromatic,light pleasant sparkling
2,Bach Extrísimo Blanco Semidulce 2021,2021,Masia Bach,9.4,0.75,White,White,Conventional,Cataluña,Catalunya,Xarel·lo,Blend,Highly aromatic / Complex / Fruit aromas / Flo...,Good entry / Structured / Balanced / Good acid...,11.5,4.8,95.0,NaN,NaN,NaN,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...,highly_aromatic complex fruit_aromas floral_sc...,good_entry structured balanced good_acidity fl...


In [182]:
vinissimus_short_exploded.to_csv("clean_grapes.csv", index=False)